# **NYC Taxi Data Analysis - Yellow Taxis**

Exploratory analysis on NYC Yellow Taxis looking into customer segmentation and factors that may impact tipping behaviors across the 5 boroughs

--- 

### **Imports & Setup**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../scripts') # imports custom scripts from the scripts directory
from data_utils import rearr_datetime_cols, remove_outliers, same_zone_perc, identify_cancelled_rides, remove_cancelled_fare_pairs, categorize_zones, neighborhood_fare_quantiles, borough_tip_cleaned, match_tip_neighborhoods, constant_tips

# setting display options for pandas
pd.set_option('display.max_columns', None) # displays all columns in the dataframe
pd.set_option('display.max_rows', 100) # sets the max number of rows to 100

### **Loading Data**

In [2]:
# look up tables
zone = pd.read_csv('../data/taxi_zone_lookup.csv') 
vendor = pd.read_csv('../data/taxi_vendor_lookup.csv')
payment = pd.read_csv('../data/payment_lookup.csv')
ratecode = pd.read_csv('../data/ratecode_lookup.csv')

# yellow taxi data
df = pd.read_parquet('../data/yellow_tripdata_2025-05.parquet')

In [3]:
df.columns  # displays all columns in the dataframe

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee',
       'cbd_congestion_fee'],
      dtype='object')

In [4]:
df.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
count,4.591845e+06,4591845,4591845,3.395669e+06,4.591845e+06,3.395669e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,4.591845e+06,3.395669e+06,3.395669e+06,4.591845e+06
mean,1.875653e+00,2025-05-16 07:15:25.300312,2025-05-16 07:33:20.003034,1.295270e+00,7.653422e+00,2.432585e+00,1.611871e+02,1.611876e+02,9.123720e-01,1.835794e+01,1.164838e+00,4.774078e-01,2.858122e+00,5.114760e-01,9.563765e-01,2.688033e+01,2.197677e+00,1.531993e-01,5.278734e-01
min,1.000000e+00,2009-01-01 00:20:39,2009-01-01 00:20:49,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-9.980000e+02,-1.739000e+01,-5.000000e-01,-9.044000e+01,-1.481700e+02,-1.000000e+00,-1.147170e+03,-2.500000e+00,-1.750000e+00,-7.500000e-01
25%,2.000000e+00,2025-05-08 18:23:12,2025-05-08 18:42:50,1.000000e+00,1.060000e+00,1.000000e+00,1.140000e+02,1.070000e+02,0.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.554000e+01,2.500000e+00,0.000000e+00,0.000000e+00
50%,2.000000e+00,2025-05-16 00:49:45,2025-05-16 01:05:55,1.000000e+00,1.890000e+00,1.000000e+00,1.610000e+02,1.620000e+02,1.000000e+00,1.415000e+01,0.000000e+00,5.000000e-01,2.000000e+00,0.000000e+00,1.000000e+00,2.142000e+01,2.500000e+00,0.000000e+00,7.500000e-01
75%,2.000000e+00,2025-05-23 13:28:04,2025-05-23 13:47:48,1.000000e+00,3.770000e+00,1.000000e+00,2.330000e+02,2.330000e+02,1.000000e+00,2.300000e+01,2.500000e+00,5.000000e-01,4.000000e+00,0.000000e+00,1.000000e+00,3.098000e+01,2.500000e+00,0.000000e+00,7.500000e-01
max,7.000000e+00,2025-06-01 00:04:31,2025-06-04 11:17:10,9.000000e+00,2.631040e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,1.583600e+03,1.336000e+02,2.214000e+01,4.432100e+02,1.481700e+02,1.000000e+00,1.614290e+03,2.500000e+00,6.750000e+00,1.250000e+00
std,7.239395e-01,NaN,NaN,7.252927e-01,6.533947e+02,1.133495e+01,6.655046e+01,7.058759e+01,7.590985e-01,1.982062e+01,1.845559e+00,1.380384e-01,4.043892e+00,2.144886e+00,2.750487e-01,2.419524e+01,9.446371e-01,5.431268e-01,3.607600e-01


In [5]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-05-01 00:07:06,2025-05-01 00:24:15,1.0,3.70,1.0,N,140,202,1,18.4,4.25,0.5,4.85,0.00,1.0,29.00,2.5,0.00,0.75
1,2,2025-05-01 00:07:44,2025-05-01 00:14:27,1.0,1.03,1.0,N,234,161,1,8.6,1.00,0.5,4.30,0.00,1.0,18.65,2.5,0.00,0.75
2,2,2025-05-01 00:15:56,2025-05-01 00:23:53,1.0,1.57,1.0,N,161,234,2,10.0,1.00,0.5,0.00,0.00,1.0,15.75,2.5,0.00,0.75
3,2,2025-05-01 00:00:09,2025-05-01 00:25:29,1.0,9.48,1.0,N,138,90,1,40.8,6.00,0.5,11.70,6.94,1.0,71.94,2.5,1.75,0.75
4,2,2025-05-01 00:45:07,2025-05-01 00:52:45,1.0,1.80,1.0,N,90,231,1,10.0,1.00,0.5,1.50,0.00,1.0,17.25,2.5,0.00,0.75


---
### **Data Assumptions**
* Each row represents a single trip record
* No explicit primary key column
* Each row's uniqueness could be composed of a combination of columns (pickup, dropoff, vendor) 
  
*Note: for this exploratory analysis, a primary key is not critical, as much of the analysis is focused on aggregation patterns rather than uniquely identifying rows* 
> If necessary, could create a 'ride_id' based off a combination of columns as a primary key

---
### **Data Cleaning**

In [6]:
# shows the number of missing values per column
df.isna().sum() 


VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count          1196176
trip_distance                  0
RatecodeID               1196176
store_and_fwd_flag       1196176
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     1196176
Airport_fee              1196176
cbd_congestion_fee             0
dtype: int64

In [7]:
# checking to see the unique values in the passenger_count column 
df['passenger_count'].unique() 

array([ 1.,  0.,  2.,  3.,  4.,  5.,  6.,  9.,  8., nan])

In [8]:
df[df['passenger_count'] == 0] # it's odd that there are 0's for passenger_count, doesn't make sense, investigate further
df['passenger_count'] = df['passenger_count'].replace({0: pd.NA, np.nan: pd.NA}) # doesn't seem like there was an actual reason for it, will replace 0s with nulls and replace np.nan with NA to standardize
df.drop(columns=['store_and_fwd_flag'], inplace=True) # dropping store_and_fwd_flag because it's not relevant for this specific analysis
df.isna().sum() # checking to see the number of missing values per column again after the changes

VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count          1220143
trip_distance                  0
RatecodeID               1196176
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     1196176
Airport_fee              1196176
cbd_congestion_fee             0
dtype: int64

In [9]:
df['RatecodeID'].unique() 
df['RatecodeID'] = df['RatecodeID'].replace(np.nan, 99) # replace nulls with 99, as listed in the source dict
df.duplicated().sum() # checking for duplicates

0

In [10]:
# checking lookup tables individually
zone.head(), payment.head(), ratecode.head(), vendor.head()

(   LocationID        Borough                     Zone service_zone
 0           1            EWR           Newark Airport          EWR
 1           2         Queens              Jamaica Bay    Boro Zone
 2           3          Bronx  Allerton/Pelham Gardens    Boro Zone
 3           4      Manhattan            Alphabet City  Yellow Zone
 4           5  Staten Island            Arden Heights    Boro Zone,
    payment_type  payment_method
 0             0  Flex Fare trip
 1             1     Credit card
 2             2            Cash
 3             3       No charge
 4             4         Dispute,
    RatecodeID                   Rate
 0           1              Standard 
 1           2                    JFK
 2           3                 Newark
 3           4  Nassau or Westchester
 4           5        Negotiated fare,
    VendorID                        vendor_name  Unnamed: 2
 0         1  Creative Mobile Technologies, LLC         NaN
 1         2                 Curb Mobility,

In [11]:
vendor = vendor.loc[:, ~vendor.columns.str.contains('^Unnamed')] # vendor has 2 unnamed columns, let's filter those out, ~ removes columns that start with 'Unnamed'

---
### **Exploratory**: The 5 boroughs of NYC

__A few notes:__

1. Borough DataFrames:
    > DataFrames named after a borough (e.g., `manhattan`, `queens`) contain only rides where **BOTH** pickup & dropoff are within that borough.
2. '_overall' DataFrames:
    > DataFrames with '_overall' (e.g., `manhattan_overall`) include any ride where **EITHER** the pickup or dropoff is in that borough.
3. '_cleaned' DataFrames:
    > DataFrames with '_cleaned' (e.g., `queens_cleaned`) have extreme outliers in trip duration **AND** distance removed. These are used for analyses that are sensitive to outliers (like distribution or clustering analysis). These are dfs are bound to have changes in other segments of this project.
4. Why Use Different DataFrames:
    > Different questions require different scopes, some analyses need clean data to avoid skewed results


In [12]:
# merging lookup tables with the taxi data (left joins because we want to retain all records from the taxi data)
merged = df.merge(zone, how='left', left_on='PULocationID', right_on='LocationID').rename(columns={'Borough': 'PU_Borough', 'Zone': 'PU_Zone', 'service_zone': 'PU_service_zone'}) # saving location information based off PULocationID 
merged = merged.merge(zone, how='left', left_on='DOLocationID', right_on='LocationID').rename(columns={'Borough': 'DO_Borough', 'Zone': 'DO_Zone', 'service_zone': 'DO_service_zone'}) # saving location information based off DOLocationID

merged = merged.drop(columns=['LocationID_x', 'LocationID_y']) # dropping LocationID columns, no longer needed
merged = merged.merge(vendor, how='left', on='VendorID').merge(payment, how='left', on='payment_type').merge(ratecode, how='left', on='RatecodeID') # merging the rest of the lookups

# create a column with the difference between pickup and dropoff datetime as 'duration_mins'
merged['duration_mins'] = (merged['tpep_dropoff_datetime'] - merged['tpep_pickup_datetime']).dt.total_seconds() / 60 # duration in minutes
merged.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,vendor_name,payment_method,Rate,duration_mins
0,1,2025-05-01 00:07:06,2025-05-01 00:24:15,1.0,3.70,1.0,140,202,1,18.4,4.25,0.5,4.85,0.00,1.0,29.00,2.5,0.00,0.75,Manhattan,Lenox Hill East,Yellow Zone,Manhattan,Roosevelt Island,Boro Zone,"Creative Mobile Technologies, LLC",Credit card,Standard,17.150000
1,2,2025-05-01 00:07:44,2025-05-01 00:14:27,1.0,1.03,1.0,234,161,1,8.6,1.00,0.5,4.30,0.00,1.0,18.65,2.5,0.00,0.75,Manhattan,Union Sq,Yellow Zone,Manhattan,Midtown Center,Yellow Zone,"Curb Mobility, LLC",Credit card,Standard,6.716667
2,2,2025-05-01 00:15:56,2025-05-01 00:23:53,1.0,1.57,1.0,161,234,2,10.0,1.00,0.5,0.00,0.00,1.0,15.75,2.5,0.00,0.75,Manhattan,Midtown Center,Yellow Zone,Manhattan,Union Sq,Yellow Zone,"Curb Mobility, LLC",Cash,Standard,7.950000
3,2,2025-05-01 00:00:09,2025-05-01 00:25:29,1.0,9.48,1.0,138,90,1,40.8,6.00,0.5,11.70,6.94,1.0,71.94,2.5,1.75,0.75,Queens,LaGuardia Airport,Airports,Manhattan,Flatiron,Yellow Zone,"Curb Mobility, LLC",Credit card,Standard,25.333333
4,2,2025-05-01 00:45:07,2025-05-01 00:52:45,1.0,1.80,1.0,90,231,1,10.0,1.00,0.5,1.50,0.00,1.0,17.25,2.5,0.00,0.75,Manhattan,Flatiron,Yellow Zone,Manhattan,TriBeCa/Civic Center,Yellow Zone,"Curb Mobility, LLC",Credit card,Standard,7.633333


In [13]:
merged.isna().sum() #after merging, there are some nulls in the lookup columns, which is expected since not all records will have a corresponding entry in the lookup tables

VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count          1220143
trip_distance                  0
RatecodeID                     0
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     1196176
Airport_fee              1196176
cbd_congestion_fee             0
PU_Borough                  2091
PU_Zone                     7596
PU_service_zone             9687
DO_Borough                 20490
DO_Zone                     9800
DO_service_zone            30290
vendor_name                    0
payment_method                 0
Rate                           0
duration_mins                  0
dtype: int64

In [14]:
merged['PU_Zone'].unique(), merged['DO_Zone'].unique(), merged['PU_Borough'].unique(), merged['DO_Borough'].unique() # checking unique values in the merged columns
merged['PU_Borough'] = merged['PU_Borough'].replace(np.nan, 'Unknown')
merged['DO_Borough'] = merged['DO_Borough'].replace(np.nan, 'Unknown') # there are 'Unknown' and 'NaN' values in PU_Borough & DO_Borough, replace na with 'Unknown' for consistency

#### **Understanding the Data (Geographic & Trip Characteristics)**
>How do often people request rides, where do they request the most?
* Focusing on location to perform aggregation

In [15]:
ride_combination = (merged.groupby(['PU_Borough', 'DO_Borough']).size().reset_index(name='count').sort_values(by='count',ascending=False)) # count occurrences of rides requested from PU_Borough and DO_Borough & sorts 
ride_combination.head() # by a good chunk, the most frequent combination is from Manhattan to Manhattan

,PU_Borough,DO_Borough,count
23,Manhattan,Manhattan,3657462
30,Queens,Manhattan,225823
31,Queens,Queens,136701
24,Manhattan,Queens,114579
8,Brooklyn,Brooklyn,104316


*As shown above:*
> The results shows Manhattan having the most ride frequency, let's investigate Manhattan a bit further:
  * Could there be a trend since most rides are within Manhattan? Consider:
    * Yellow taxis could be commonly used for short trips strictly within Manhattan 
    * Take a closer look at the datetime & trip distance to confirm the durations of these rides with its corresponding distance


In [16]:
# Manhattan ONLY (PU_Borough & DO_Borough both equal 'Manhattan')
manhattan = merged[(merged['PU_Borough'] == 'Manhattan') & (merged['DO_Borough'] == 'Manhattan')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})

# reindexing to have the columns in a more readable order
manhattan_datetime_cols = ['PU_datetime', 'DO_datetime', 'duration_mins', 'trip_distance', 'passenger_count', 'PU_Zone', 'PU_service_zone', 'DO_Zone', 'DO_service_zone']
manhattan = manhattan[manhattan_datetime_cols + [c for c in manhattan.columns if c not in manhattan_datetime_cols]] 
manhattan.head() # checking after reindexing

,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
0,2025-05-01 00:07:06,2025-05-01 00:24:15,17.150000,3.70,1.0,Lenox Hill East,Yellow Zone,Roosevelt Island,Boro Zone,1,1.0,140,202,1,18.4,4.25,0.5,4.85,0.0,1.0,29.00,2.5,0.0,0.75,Manhattan,Manhattan,"Creative Mobile Technologies, LLC",Credit card,Standard
1,2025-05-01 00:07:44,2025-05-01 00:14:27,6.716667,1.03,1.0,Union Sq,Yellow Zone,Midtown Center,Yellow Zone,2,1.0,234,161,1,8.6,1.00,0.5,4.30,0.0,1.0,18.65,2.5,0.0,0.75,Manhattan,Manhattan,"Curb Mobility, LLC",Credit card,Standard
2,2025-05-01 00:15:56,2025-05-01 00:23:53,7.950000,1.57,1.0,Midtown Center,Yellow Zone,Union Sq,Yellow Zone,2,1.0,161,234,2,10.0,1.00,0.5,0.00,0.0,1.0,15.75,2.5,0.0,0.75,Manhattan,Manhattan,"Curb Mobility, LLC",Cash,Standard
4,2025-05-01 00:45:07,2025-05-01 00:52:45,7.633333,1.80,1.0,Flatiron,Yellow Zone,TriBeCa/Civic Center,Yellow Zone,2,1.0,90,231,1,10.0,1.00,0.5,1.50,0.0,1.0,17.25,2.5,0.0,0.75,Manhattan,Manhattan,"Curb Mobility, LLC",Credit card,Standard
6,2025-05-01 00:18:14,2025-05-01 00:27:38,9.400000,1.50,<NA>,Lenox Hill East,Yellow Zone,Yorkville West,Yellow Zone,1,1.0,140,263,1,11.4,3.50,0.5,4.05,0.0,1.0,20.45,2.5,0.0,0.00,Manhattan,Manhattan,"Creative Mobile Technologies, LLC",Credit card,Standard


> Finding the % of manhattan to manhattan rides that have short durations or trip distances
* take a look at the distribution of trip durations & distances 
* filter out negative value into 'manhattan_cleaned', likely input error 

In [17]:
manhattan_cleaned = manhattan[manhattan['duration_mins'] >= 0]
manhattan_cleaned[['duration_mins', 'trip_distance']].describe(), merged[['trip_distance', 'duration_mins']].describe()

(       duration_mins  trip_distance
 count   3.657438e+06   3.657438e+06
 mean    1.406173e+01   4.643766e+00
 std     2.414163e+01   5.236403e+02
 min     0.000000e+00   0.000000e+00
 25%     7.550000e+00   9.500000e-01
 50%     1.208333e+01   1.580000e+00
 75%     1.816667e+01   2.560000e+00
 max     7.664717e+03   2.631040e+05,
        trip_distance  duration_mins
 count   4.591845e+06   4.591845e+06
 mean    7.653422e+00   1.791171e+01
 std     6.533947e+02   2.799913e+01
 min     0.000000e+00  -6.810833e+02
 25%     1.060000e+00   8.366667e+00
 50%     1.890000e+00   1.391667e+01
 75%     3.770000e+00   2.216667e+01
 max     2.631040e+05   7.664717e+03)

In [18]:
merged_cleaned = merged[merged['duration_mins'] >= 0] # let's also clean the merged df to remove negative values

# we can use values <= the 25th percentile of the overall df to define short rides by durations OR distances
duration_25_all = merged_cleaned['duration_mins'].quantile(0.25)
distance_25_all = merged_cleaned['trip_distance'].quantile(0.25)

# defining short rides in Manhattan (all rides with short durations OR distances)
m_short_rides = manhattan_cleaned[(manhattan_cleaned['duration_mins'] <= duration_25_all) | (manhattan_cleaned['trip_distance'] <= distance_25_all)] 
m_short_rides_percentage = (m_short_rides.shape[0] / manhattan_cleaned.shape[0]) * 100
print(f"% of short rides in Manhattan (by duration or distance): {m_short_rides_percentage:.2f}%")

% of short rides in Manhattan (by duration or distance): 38.83%


>It doesn't necessarily seem like yellow taxis are used mainly for short trips within Manhattan
* Look from a broader perspective to display zones pairs where rides are frequently being called from and to

In [19]:
# rides involving Manhattan (PU_Borough or DO_Borough equal 'Manhattan')
manhattan_overall = merged[(merged['PU_Borough'] == 'Manhattan') | (merged['DO_Borough'] == 'Manhattan')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
manhattan_overall.groupby(['PU_Zone', 'DO_Zone']).size().reset_index(name='count').sort_values(by='count', ascending=False).head()  

,PU_Zone,DO_Zone,count
18782,Upper East Side South,Upper East Side North,29651
18547,Upper East Side North,Upper East Side South,25399
18783,Upper East Side South,Upper East Side South,20890
18546,Upper East Side North,Upper East Side North,19259
12306,Midtown Center,Upper East Side South,13813


>Could be useful to look into Same-Zone Rides in Manhattan vs. Manhattan-Involved Rides

In [20]:
# checking the amount of rides within the same zone for ride within Manhattan (PU_Borough & DO_Borough equals Manhattan, AND PU_Zone == DO_Zone)
m_same_zone_percentage = (manhattan[manhattan['PU_Zone'] == manhattan['DO_Zone']].shape[0]/manhattan.shape[0])*100  # percentage of rides within the same zone in Manhattan
print(f"% of same zone rides within Manhattan: {m_same_zone_percentage:.2f}%")

% of same zone rides within Manhattan: 4.70%


>It's a low percentage:
  * Maybe there are less local taxi rides because the borough is more walkable?
    * The numbers are on the lower end for duration & distance
    * Could indicate most taxi rides are cross-neighborhood or intra-borough
    * Will compare with other boroughs to see if Manhattan is unique in this behavior

In [21]:
# checking the amount of rides within the same zone for all Manhattan-involved rides (PU_Borough | DO_Borough equals 'Manhattan' AND, PU_Zone == DO_Zone)
m_same_zone_percentage = (manhattan[manhattan['PU_Zone'] == manhattan['DO_Zone']].shape[0]/manhattan_overall.shape[0])*100  # using manhattan_overall instead
print(f"% of same zone rides involving Manhattan: {m_same_zone_percentage:.2f}%")

% of same zone rides involving Manhattan: 4.09%


> The percentage is low for both, and even lower when considering all Manhattan-involved rides
* This demonstrates that most yellow taxi rides in Manhattan are not hyper-local, but instead cross zones or neighborhoods
* Could support the idea that Manhattan's walkability reduces the need for taxis for very short and local trips due to the borough being less spaced out

> *Note: I'll be saving a few functions from above in the data_utils pyscript since some will be applied to the rest of the boroughs*
* **'rearr_datetime_cols':**
  * referring to the reindexing/reordering of columns to have the dataframe in a readable manner
* **'remove_outliers':**
  * filters negative values out for 'duration_mins' & 'trip_distance'
  * then finds the 1 and 99th quantile based off those 2 columns and removes those values
* **'same_zone_perc':**
  * returns the percentage of rides within the same zone strictly and involving the borough

>Analyzing the rest of the boroughs: Queens

In [22]:
# ONLY Queens (PU_Borough and DO_Borough both equal 'Queens')
queens = merged[(merged['PU_Borough'] == 'Queens') & (merged['DO_Borough'] == 'Queens')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
queens = rearr_datetime_cols(queens)
queens.head()

,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
5,2025-05-01 00:09:24,2025-05-01 00:22:04,12.666667,5.11,1.0,LaGuardia Airport,Airports,Sunnyside,Boro Zone,2,1.0,138,226,1,22.6,6.00,0.5,6.02,0.0,1.0,37.87,0.0,1.75,0.0,Queens,Queens,"Curb Mobility, LLC",Credit card,Standard
54,2025-05-01 00:39:32,2025-05-01 01:13:48,34.266667,16.51,1.0,JFK Airport,Airports,JFK Airport,Airports,2,1.0,132,132,1,66.0,1.00,0.5,17.12,0.0,1.0,85.62,0.0,0.00,0.0,Queens,Queens,"Curb Mobility, LLC",Credit card,Standard
126,2025-05-01 00:11:11,2025-05-01 00:24:54,13.716667,4.53,2.0,JFK Airport,Airports,Richmond Hill,Boro Zone,2,1.0,132,197,1,20.5,1.00,0.5,6.90,0.0,1.0,31.65,0.0,1.75,0.0,Queens,Queens,"Curb Mobility, LLC",Credit card,Standard
148,2025-05-01 00:29:36,2025-05-01 01:06:47,37.183333,9.18,2.0,JFK Airport,Airports,Hollis,Boro Zone,2,1.0,132,122,2,43.6,1.00,0.5,0.00,0.0,1.0,47.85,0.0,1.75,0.0,Queens,Queens,"Curb Mobility, LLC",Cash,Standard
172,2025-05-01 00:13:08,2025-05-01 00:30:13,17.083333,7.20,1.0,LaGuardia Airport,Airports,Kew Gardens,Boro Zone,1,1.0,138,134,2,29.6,7.75,0.5,0.00,0.0,1.0,38.85,0.0,1.75,0.0,Queens,Queens,"Creative Mobile Technologies, LLC",Cash,Standard


> It seems like there's a lot of service for airports (LaGuardia & JFK) strictly within Queens
* Worth noting that both airports are located in Queens, could explain the high volume of airport type services

In [23]:
# checking the amount of rides in each service_zone
q_PU_service_zones = queens['PU_service_zone'].value_counts().rename('PU_count')  
q_DO_service_zones = queens['DO_service_zone'].value_counts().rename('DO_count')  
pd.concat([q_PU_service_zones, q_DO_service_zones], axis=1) # side-by-side counts of PU and DO service zones for Queens

,PU_count,DO_count
Boro Zone,70585,114926
Airports,66116,21775


> Looks like there's more airport services at pickup compared to dropoff

In [24]:
# rides involving Queens (PU_Borough or DO_Borough equal 'Queens')
queens_overall = merged[(merged['PU_Borough'] == 'Queens') | (merged['DO_Borough'] == 'Queens')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})

# the percentage of rides that are airport-related
q_airport_rides = queens_overall[queens_overall['PU_service_zone'].str.contains('Airport') | queens_overall['DO_service_zone'].str.contains('Airport')].shape[0]
q_airport_percentage = (q_airport_rides / queens_overall.shape[0]) * 100
print(f"% of airport-related rides in queens: {q_airport_percentage:.2f}%")

% of airport-related rides in queens: 67.23%


In [25]:
# compared to manhattan, Queens has a higher percentage of airport-related rides
m_airport_rides = manhattan_overall[manhattan_overall['PU_service_zone'].str.contains('Airport') | manhattan_overall['DO_service_zone'].str.contains('Airport')].shape[0]
m_airport_percentage = (m_airport_rides / manhattan_overall.shape[0]) * 100
print(f"% of airport-related rides in manhattan: {m_airport_percentage:.2f}%")

% of airport-related rides in manhattan: 6.20%


> In total, over half the rides involving queens are airport-related
* Could be that queens uses taxi services for lesser casual means, potentially longer trips

In [26]:
# let's filter out the extreme outliers in both columns
queens_cleaned = remove_outliers(queens)
queens_cleaned[['duration_mins', 'trip_distance']].describe()

,duration_mins,trip_distance
count,120305.000000,120305.000000
mean,18.218662,5.671123
std,11.372003,3.965334
min,0.100000,0.010000
25%,10.383333,2.430000
50%,16.233333,4.900000
75%,23.983333,8.480000
max,61.050000,16.570000


*As shown above:*
* The duration is much higher than that of Manhattan, which could be due to the fact that Queens is less densely populated and has longer distances between locations (less "walkable")
* Confirms taxi services are used longer trips, possibly for airport-related services or other longer-distance travel needs

>Analyzing the rest of the boroughs: Brooklyn

In [27]:
# ONLY Brooklyn (PU_Borough & DO_Borough both equal 'Brooklyn')
brooklyn = merged[(merged['PU_Borough'] == 'Brooklyn') & (merged['DO_Borough'] == 'Brooklyn')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
# rides involving Brooklyn (PU_Borough or DO_Borough equal 'Brooklyn')
brooklyn_overall = merged[(merged['PU_Borough'] == 'Brooklyn') | (merged['DO_Borough'] == 'Brooklyn')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
brooklyn = rearr_datetime_cols(brooklyn) # reindexing to have the columns in a more readable order
brooklyn.head()

,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
266,2025-05-01 00:00:07,2025-05-01 00:22:57,22.833333,3.91,1.0,Boerum Hill,Boro Zone,Sunset Park West,Boro Zone,2,1.0,25,228,2,24.0,1.0,0.5,0.00,0.0,1.0,26.50,0.0,0.0,0.0,Brooklyn,Brooklyn,"Curb Mobility, LLC",Cash,Standard
313,2025-05-01 00:28:24,2025-05-01 00:34:01,5.616667,1.00,1.0,Williamsburg (South Side),Boro Zone,Williamsburg (South Side),Boro Zone,1,1.0,256,256,1,7.2,1.0,0.5,4.00,0.0,1.0,13.70,0.0,0.0,0.0,Brooklyn,Brooklyn,"Creative Mobile Technologies, LLC",Credit card,Standard
484,2025-05-01 00:50:08,2025-05-01 00:56:34,6.433333,1.03,1.0,Williamsburg (North Side),Boro Zone,East Williamsburg,Boro Zone,2,1.0,255,80,1,7.9,1.0,0.5,3.12,0.0,1.0,13.52,0.0,0.0,0.0,Brooklyn,Brooklyn,"Curb Mobility, LLC",Credit card,Standard
567,2025-05-01 00:12:59,2025-05-01 00:26:24,13.416667,2.80,1.0,Prospect Park,Boro Zone,Flatbush/Ditmas Park,Boro Zone,2,1.0,190,89,1,15.6,1.0,0.5,3.62,0.0,1.0,21.72,0.0,0.0,0.0,Brooklyn,Brooklyn,"Curb Mobility, LLC",Credit card,Standard
760,2025-05-01 00:54:25,2025-05-01 01:10:29,16.066667,4.51,1.0,Williamsburg (North Side),Boro Zone,Carroll Gardens,Boro Zone,2,1.0,255,40,1,21.9,1.0,0.5,4.88,0.0,1.0,29.28,0.0,0.0,0.0,Brooklyn,Brooklyn,"Curb Mobility, LLC",Credit card,Standard


In [28]:
# let's filter out the extreme outliers in both columns
brooklyn_cleaned = remove_outliers(brooklyn)
brooklyn_cleaned[['duration_mins', 'trip_distance']].describe()

,duration_mins,trip_distance
count,94404.000000,94404.000000
mean,20.098065,3.424336
std,10.882119,2.247111
min,2.316667,0.010000
25%,12.150000,1.800000
50%,18.133333,3.000000
75%,25.833333,4.440000
max,65.250000,13.200000


> It seems within Brooklyn, the duration and distance is longer than Manhattan, but slightly similar to Queens
* Look into most common pickup and dropoff zones
* Short vs. long rides

In [29]:
# checking the top paired zones in Brooklyn
brooklyn.groupby(['PU_Zone','DO_Zone']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).head()  

,PU_Zone,DO_Zone,counts
1570,East New York,East New York,655
1098,Crown Heights North,Crown Heights North,434
2652,Park Slope,Park Slope,403
747,Canarsie,Canarsie,400
2120,Greenpoint,Greenpoint,349


In [30]:
same_zone_perc(brooklyn, brooklyn_overall, 'Brooklyn') # numbers seem small, checking the percentage of rides within the same zone for rides strictly within Brooklyn

% of same zone rides within Brooklyn: 7.45%
% of same zone rides involving Brooklyn: 2.25%


> Could be that Brooklyn is a borough that's considered "walkable" due to the low % 
* But if that's the case, why are the numbers so different than Manhattan?
  * Most of the rides may be for intra-borough trips, perhaps Brooklyn's neighborhoods are spaced further apart due to the larger numbers
    * Compare short rides vs. long rides in Brooklyn
    * We can use the same 25th percentile values for duration and distance as we did for Manhattan


In [31]:
# defining short rides in Brooklyn
b_short_rides = brooklyn_cleaned[(brooklyn_cleaned['duration_mins'] <= duration_25_all) | (brooklyn_cleaned['trip_distance'] <= distance_25_all)] # all rides with short durations or distances
b_short_rides_percentage = (b_short_rides.shape[0] / brooklyn_cleaned.shape[0]) * 100
print(f"% of short rides in Brooklyn (by duration or distance): {b_short_rides_percentage:.2f}%")

# defining long rides in Brooklyn
duration_75_all = merged['duration_mins'].quantile(0.75)
distance_75_all = merged['trip_distance'].quantile(0.75)

# using & instead of | to define long rides to take into account impact of delays and traffic
b_long_rides = brooklyn_cleaned[(brooklyn_cleaned['duration_mins'] >= duration_75_all) & (brooklyn_cleaned['trip_distance'] >= distance_75_all)] # all rides with long durations and distances
b_long_rides_percentage = (b_long_rides.shape[0] / brooklyn_cleaned.shape[0]) * 100
print(f"% of long rides in Brooklyn (by duration and distance): {b_long_rides_percentage:.2f}%")

% of short rides in Brooklyn (by duration or distance): 14.74%
% of long rides in Brooklyn (by duration and distance): 25.79%


In [32]:
# compared to Manhattan, Brooklyn has a higher percentage of longer rides
m_long_rides = manhattan_cleaned[(manhattan_cleaned['duration_mins'] >= duration_75_all) & (manhattan_cleaned['trip_distance'] >= distance_75_all)] # all rides with long durations and distances
m_long_rides_percentage = (m_long_rides.shape[0] / manhattan_cleaned.shape[0]) * 100
print(f"% of long rides in Manhattan (by duration and distance): {m_long_rides_percentage:.2f}%")

% of long rides in Manhattan (by duration and distance): 6.67%


> Brooklyn having a bigger % of longer rides show that, on average, it's likely to be more spaced out and less "walkable" compared to Manhattan. The smaller % of short rides also reinforces that taxi trips are likely to be longer than Manhattan.

In [33]:
# checking for non airport-related rides in Brooklyn
b_is_airport = (brooklyn_overall['PU_service_zone'].str.contains('Airport') | brooklyn_overall['DO_service_zone'].str.contains('Airport'))
non_airport_rides = brooklyn_overall[~b_is_airport]  # number of rides that are not airport-related
print(f"% of non-airport rides in Brooklyn: {non_airport_rides.shape[0]/brooklyn_overall.shape[0] * 100:.2f}%")

% of non-airport rides in Brooklyn: 85.81%


>Analyzing the rest of the boroughs: Bronx

In [34]:
# ONLY Bronx (PU_Borough & DO_Borough both equal 'Bronx')
bronx = merged[(merged['PU_Borough'] == 'Bronx') & (merged['DO_Borough'] == 'Bronx')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
bronx = rearr_datetime_cols(bronx) # reindexing to have the columns in a more readable order
bronx.head()

,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
3809,2025-05-01 02:53:48,2025-05-01 02:53:58,0.166667,0.02,1.0,West Concourse,Boro Zone,West Concourse,Boro Zone,2,5.0,247,247,4,-42.2,-1.0,-0.5,0.0,0.0,-1.0,-47.95,-2.5,0.0,-0.75,Bronx,Bronx,"Curb Mobility, LLC",Dispute,Negotiated fare
3810,2025-05-01 02:53:48,2025-05-01 02:53:58,0.166667,0.02,1.0,West Concourse,Boro Zone,West Concourse,Boro Zone,2,5.0,247,247,4,42.2,1.0,0.5,0.0,0.0,1.0,47.95,2.5,0.0,0.75,Bronx,Bronx,"Curb Mobility, LLC",Dispute,Negotiated fare
4388,2025-05-01 04:58:21,2025-05-01 05:08:25,10.066667,4.28,1.0,Hunts Point,Boro Zone,Mount Hope,Boro Zone,2,1.0,126,169,1,18.4,1.0,0.5,0.0,0.0,1.0,20.90,0.0,0.0,0.00,Bronx,Bronx,"Curb Mobility, LLC",Credit card,Standard
6589,2025-05-01 06:56:33,2025-05-01 08:05:18,68.750000,9.20,1.0,Morrisania/Melrose,Boro Zone,Van Nest/Morris Park,Boro Zone,1,99.0,167,242,1,27.5,0.0,0.5,0.0,0.0,0.0,28.00,0.0,0.0,0.00,Bronx,Bronx,"Creative Mobile Technologies, LLC",Credit card,Null/Unknown
6856,2025-05-01 06:49:55,2025-05-01 07:44:07,54.200000,6.30,1.0,Morrisania/Melrose,Boro Zone,Norwood,Boro Zone,1,99.0,167,174,1,28.5,0.0,0.5,0.0,0.0,0.0,29.00,0.0,0.0,0.00,Bronx,Bronx,"Creative Mobile Technologies, LLC",Credit card,Null/Unknown


In [35]:
# let's filter out the extreme outliers in both columns
bronx_cleaned = remove_outliers(bronx)
bronx_cleaned[['duration_mins', 'trip_distance']].describe()

,duration_mins,trip_distance
count,17419.000000,17419.000000
mean,16.286914,3.738788
std,7.685055,2.255315
min,2.233333,0.020000
25%,10.916667,1.900000
50%,15.116667,3.360000
75%,20.283333,5.190000
max,47.166667,10.710000


In [36]:
# top 10 most frequent ride combinations strictly within Bronx
bronx_cleaned.groupby(['PU_Zone', 'DO_Zone']).size().reset_index(name='count').sort_values(by='count', ascending=False).head(10)  

,PU_Zone,DO_Zone,count
800,Mott Haven/Port Morris,West Concourse,60
782,Mott Haven/Port Morris,Melrose South,58
704,Melrose South,Mott Haven/Port Morris,58
803,Mott Haven/Port Morris,Williamsbridge/Olinville,57
1167,Soundview/Castle Hill,Soundview/Castle Hill,56
784,Mott Haven/Port Morris,Mott Haven/Port Morris,54
775,Mott Haven/Port Morris,East Tremont,54
774,Mott Haven/Port Morris,East Concourse/Concourse Village,53
785,Mott Haven/Port Morris,Mount Hope,51
742,Morrisania/Melrose,Melrose South,51


In [37]:
# rides involving Bronx (PU_Borough or DO_Borough equal 'Bronx')
bronx_overall = merged[(merged['PU_Borough'] == 'Bronx') | (merged['DO_Borough'] == 'Bronx')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
bronx_overall.groupby(['PU_Zone', 'DO_Zone']).size().reset_index(name='count').sort_values(by='count', ascending=False).head(10)  

,PU_Zone,DO_Zone,count
4234,JFK Airport,Spuyten Duyvil/Kingsbridge,227
4230,JFK Airport,Riverdale/North Riverdale/Fieldston,207
4223,JFK Airport,Mott Haven/Port Morris,204
4587,LaGuardia Airport,Spuyten Duyvil/Kingsbridge,178
4576,LaGuardia Airport,Mott Haven/Port Morris,167
4213,JFK Airport,East Concourse/Concourse Village,158
5890,Mott Haven/Port Morris,Mott Haven/Port Morris,153
4231,JFK Airport,Schuylerville/Edgewater Park,147
5791,Mott Haven/Port Morris,Central Harlem North,146
4583,LaGuardia Airport,Riverdale/North Riverdale/Fieldston,139


In [38]:
same_zone_perc(bronx, bronx_overall, 'Bronx') # checking the percentage of rides within the same zone for rides strictly within Bronx

% of same zone rides within Bronx: 6.60%
% of same zone rides involving Bronx: 1.74%


In [39]:
bronx_overall['PU_service_zone'].value_counts() | bronx_overall['DO_service_zone'].value_counts()  # not much airport-related rides in Bronx

PU_service_zone
Boro Zone      117587
Yellow Zone     15615
Airports         5755
EWR                 5
Name: count, dtype: int64

>Analyzing the rest of the boroughs: Staten Island

In [40]:
# ONLY Staten Island (PU_Borough & DO_Borough both equal 'Staten Island')
staten_island = merged[(merged['PU_Borough'] == 'Staten Island') & (merged['DO_Borough'] == 'Staten Island')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
staten_island = rearr_datetime_cols(staten_island) # reindexing to have the columns in a more readable order
staten_island.head()

,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
161085,2025-05-02 12:32:44,2025-05-02 12:45:14,12.500000,4.51,1.0,Bloomfield/Emerson Hill,Boro Zone,Mariners Harbor,Boro Zone,2,1.0,23,156,2,19.8,0.0,0.5,0.0,0.0,1.0,21.3,0.0,0.0,0.0,Staten Island,Staten Island,"Curb Mobility, LLC",Cash,Standard
185265,2025-05-02 16:05:37,2025-05-02 16:49:46,44.150000,9.01,1.0,Saint George/New Brighton,Boro Zone,Heartland Village/Todt Hill,Boro Zone,2,1.0,206,118,1,47.1,2.5,0.5,10.0,0.0,1.0,61.1,0.0,0.0,0.0,Staten Island,Staten Island,"Curb Mobility, LLC",Credit card,Standard
205941,2025-05-02 18:45:29,2025-05-02 18:52:10,6.683333,1.20,1.0,Stapleton,Boro Zone,Stapleton,Boro Zone,1,1.0,221,221,2,7.9,2.5,0.5,0.0,0.0,1.0,11.9,0.0,0.0,0.0,Staten Island,Staten Island,"Creative Mobile Technologies, LLC",Cash,Standard
258868,2025-05-03 05:19:45,2025-05-03 05:19:52,0.116667,0.00,1.0,South Beach/Dongan Hills,Boro Zone,South Beach/Dongan Hills,Boro Zone,2,5.0,214,214,2,80.0,0.0,0.5,0.0,0.0,1.0,81.5,0.0,0.0,0.0,Staten Island,Staten Island,"Curb Mobility, LLC",Cash,Negotiated fare
258869,2025-05-03 05:20:56,2025-05-03 05:21:00,0.066667,0.00,1.0,South Beach/Dongan Hills,Boro Zone,South Beach/Dongan Hills,Boro Zone,2,5.0,214,214,2,82.0,0.0,0.5,0.0,0.0,1.0,83.5,0.0,0.0,0.0,Staten Island,Staten Island,"Curb Mobility, LLC",Cash,Negotiated fare


In [41]:
# there's not much trip data within Staten Island
staten_island.shape[0]

139

In [42]:
# rides involving Staten Island (PU_Borough or DO_Borough equal 'Staten Island')
staten_island_overall = merged[(merged['PU_Borough'] == 'Staten Island') | (merged['DO_Borough'] == 'Staten Island')].rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'})
staten_island_overall.shape[0]  # checking the number of rides involving Staten Island

1732

> There are more rides that are inter-borough vs. within borough

In [43]:
# let's filter out the extreme outliers in both columns
staten_island_cleaned = remove_outliers(staten_island)
staten_island_cleaned[['duration_mins', 'trip_distance']].describe()

,duration_mins,trip_distance
count,112.000000,112.000000
mean,11.154911,2.859464
std,6.803384,1.945837
min,0.966667,0.040000
25%,6.512500,1.330000
50%,9.666667,2.210000
75%,15.120833,4.105000
max,36.966667,8.110000


In [44]:
# comparing to the overall Staten Island rides
staten_island_overall_cleaned = remove_outliers(staten_island_overall)
staten_island_overall_cleaned[['duration_mins', 'trip_distance']].describe()

,duration_mins,trip_distance
count,1611.000000,1611.000000
mean,44.503311,17.945034
std,22.641577,8.282682
min,3.033333,0.760000
25%,29.516667,11.900000
50%,42.166667,18.490000
75%,57.025000,24.770000
max,126.566667,35.500000


In [45]:
# checking the distribution of dropoff boroughs for Staten Island rides
staten_island_overall['DO_Borough'].value_counts()

DO_Borough
Staten Island    1367
Brooklyn          264
Manhattan          78
Queens             19
Bronx               3
Unknown             1
Name: count, dtype: int64

> Compared to the rest of the boroughs, it has the shortest in terms of duration and distance within the borough. But when we take a look at the inter-borough trips, it increases 10x

> Additionally, much of the drop off points are to Staten Island
* Could be related to more traffic the closer to the center of the city or bridges

>#### **Key Findings:**
1. Manhattan has the most ride frequency out of all the 5 boroughs, followed by Queens, Brooklyn, Bronx, Staten Island in that order

2. Most yellow taxi rides in Manhattan are not hyper-local, but instead cross zones or neighborhoods. Instead of being commonly used for short trips within Manhattan, the borough is naturally closer together in distance compared to the rest of the boroughs 

3. Brooklyn & Queens seem to have longer trips than Manhattan, could be geographically more spaced out, whereas Manhattan seems more “walkable”. Therefore, Manhattan's “walkability” reduces the need for taxis for very short, local trips

4. There’s a high volume of airport type services within Queens due to JFK & LaGuardia being located there, taxis are most likely used for less casual means than that of Manhattan

5. As a whole, Taxi services seem to be less popular in Bronx, and especially in Staten Island 

6. Potential relationships with the mismatch in duration and distance to traffic, factors such as being closer to the city, or the major bridges 



---
**Note:** *Before going into analysis with fares & tips, it's important to note that there may be values that are negative, some might just be cancelled fares which should have a corresponding duplicate positive value*
> From here on out:
* **'_cleaned'** will be filtered out of cancelled rides AND unrelated negative fares to be used for aggregation as a whole for less skewed data and more accurate comparisons for fares, tips, and fees
  
* Will be **dropping 'EWR' columns** since fare & tip analysis is solely on the 5 boroughs, we will keep 'Unknown' columns since we can't rule out that the data could be potentially from the 5 boroughs or not, but it will be excluded from borough-level comparisons
* Referring to zones and neighborhood **interchangeably**

---
#### **1. Location Segmentation & Tipping Behavior**  


> Looking at neighborhoods by boroughs 
* An overview of fare amounts (as a whole & individual boroughs)
* Recall the borough dataframes from above, they'll be repurposed for the use of discovering tipping behavior for rest of the segments

*Note: Using functions in the data_utils pyscript*
* **'identify_cancelled_rides'**
  * it filters for negative and positive fares individually, then merges together on a combination of columns that serves as a unique identifier for each ride
    * 'ride_id_cols' & 'fee_cols' cannot be combined in this function because they need to be treated as different entities due to the matching pairs on the negative and postive values of the monetary columns
  * then takes the fare amount and matches to its opposite pair (the same value but negative & positive)
* **'remove_cancelled_fare_pairs'**
  * takes the matched pairs and removes it from the dataframe, along with any remaining negative fares not associated with a positive pair

In [46]:
# filtering out the canceled/refunded rides to get accurate aggregations
merged.rename(columns={'tpep_pickup_datetime': 'PU_datetime', 'tpep_dropoff_datetime': 'DO_datetime'}, inplace=True) # renaming the columns for the functions to work properly

# droping all rows where borough is 'EWR' since this analysis is focused on the 5 boroughs
merged = merged[(merged['PU_Borough'] != 'EWR') & (merged['DO_Borough'] != 'EWR')] # will keep 'Unknown' columns, but it will be excluded from borough-level comparisons

In [47]:
# identifying cancelled rides: these are rides with negative fare amounts that have corresponding positive fare amounts
#'ride_id_cols' serve as the unique ride identification
ride_id_cols = ['VendorID','PU_datetime', 'DO_datetime', 'PULocationID', 'DOLocationID', 'duration_mins', 'trip_distance', 'passenger_count', 'payment_type', 'RatecodeID']

# fare_amount and other monetary columns will be stored in 'fee_cols'
fee_cols = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'improvement_surcharge', 'congestion_surcharge', 'tolls_amount', 'Airport_fee', 'cbd_congestion_fee']

overall_canceled_rides = identify_cancelled_rides(merged, ride_id_cols, fee_cols) 
overall_canceled_rides[['fare_amount_neg', 'fare_amount_pos']].nunique() # should only return the same amount of records for each column

fare_amount_neg    803
fare_amount_pos    803
dtype: int64

In [48]:
merged_cleaned = remove_cancelled_fare_pairs(merged, overall_canceled_rides) # filtering out cancelled rides from the merged df

In [49]:
merged_cleaned['fare_amount'].mode()  # most common fare amount: $8.6

0    8.6
Name: fare_amount, dtype: float64

In [50]:
merged_cleaned['fare_amount'].mean()  # overall average fare amount: $20.30

20.302917066838795

> Using the average & quantile as a baseline or 'threshold' for fares categorized as being cheaper or pricier

In [51]:
# aggregating fare amounts by boroughs
merged_cleaned.groupby(['PU_Borough', 'DO_Borough']).agg({'fare_amount': ['mean', 'median']}).reset_index().sort_values(by=('fare_amount', 'mean'), ascending=False).head()

PU_Borough     DO_Borough fare_amount       
                                        mean median
29  Staten Island        Unknown  260.600000  260.6
23         Queens        Unknown  114.797088   98.2
34        Unknown  Staten Island  112.340000  101.0
22         Queens  Staten Island  102.913577   99.8
4           Bronx  Staten Island   93.900000   99.3

> The mean and median are relatively similar, showing that the data isn't skewed by a lot, we can continue with looking at how each borough compares to the average pricing as a whole

> The fare prices seem to be mainly based off of geographical distance, which makes sense
* There may not be much to gather on whether there are trends in pricing based off different PU and DO boroughs (inter-borough), which is why we'll look into results strictly within the specific borough

In [52]:
# categorizing the fare amounts into bins to decide which boroughs are above or below average
overall_avg_fare = merged_cleaned['fare_amount'].mean() # using the overall mean to define the bins

# calculating average fare amounts for each borough
borough_fares = (
    merged_cleaned[merged_cleaned['PU_Borough'] == merged_cleaned['DO_Borough']]
    .groupby(['PU_Borough', 'DO_Borough'])['fare_amount']
    .mean()
    .reset_index(name='average_fare')
    )
borough_fares['overall_avg_fare'] = overall_avg_fare  # adding overall average fare column for easier comparison

bins = [0, overall_avg_fare, merged_cleaned['fare_amount'].max()] # essentially: (0, overall average fare] (overall average fare, the max fare amount]
labels = ['Below Average', 'Above Average']

borough_fares['fare_category'] = pd.cut(borough_fares['average_fare'], bins=bins, labels=labels, right=True)
borough_fares[(borough_fares['PU_Borough'] != 'Unknown') & (borough_fares['DO_Borough'] != 'Unknown')][['PU_Borough', 'DO_Borough', 'average_fare', 'overall_avg_fare','fare_category']].drop_duplicates().sort_values(by='average_fare', ascending=False).head()

,PU_Borough,DO_Borough,average_fare,overall_avg_fare,fare_category
3,Queens,Queens,30.682246,20.302917,Above Average
4,Staten Island,Staten Island,29.237826,20.302917,Above Average
1,Brooklyn,Brooklyn,20.360681,20.302917,Above Average
0,Bronx,Bronx,18.786374,20.302917,Below Average
2,Manhattan,Manhattan,14.900289,20.302917,Below Average


> It seems like the pricier fares take place in Queens and Staten Island. Because we are focusing on only the 5 boroughs, EWR & Unknowns will be disregarded

> Brooklyn & Bronx seem to be closer to the average, while Manhattan is on the cheaper side

* This backs up the geographical data of Queens and Staten Island being more spaced out 
* Lets look even closer into specific zones within the boroughs to look at the varying fare amounts

In [53]:
# starting with Manhattan: using the same process as above of filtering out the canceled/refunded rides 
m_canceled_rides = identify_cancelled_rides(manhattan, ride_id_cols, fee_cols)
manhattan_cleaned = remove_cancelled_fare_pairs(manhattan, m_canceled_rides)

*Note: I'll be using a function saved in the data_utils pyscript*
* **'categorize_zones':**
  * categorizes zones based on average fare amounts and their quartiles, returns 3 dataframes: 
    * **pricey_zones**: zones with fares above the 75th percentile of borough average fare
    * **cheap_zones**: zones with fares below the 25th percentile of borough average fare
    * **avg_zones**: zones with fares between the 25th and 75th percentiles of borough average fare 

In [54]:
m_avg_fare = manhattan_cleaned['fare_amount'].mean() # overall average fare amount in Manhattan
m_pu_avg_fare = manhattan_cleaned.groupby('PU_Zone')['fare_amount'].mean().reset_index(name='avg_fare').sort_values(by='avg_fare', ascending=False) # average fare amount by PU
m_do_avg_fare = manhattan_cleaned.groupby('DO_Zone')['fare_amount'].mean().reset_index(name='avg_fare').sort_values(by='avg_fare', ascending=False) # average fare amount by DO

In [55]:
# using the categorize_zones function to categorize the zones based on average fare amounts and their quartiles
m_pricier_zones = categorize_zones(m_pu_avg_fare, m_do_avg_fare, m_avg_fare)[0] # expensive neighborhoods
m_cheap_zones = categorize_zones(m_pu_avg_fare, m_do_avg_fare, m_avg_fare)[1] # cheap neighborhoods
m_avg_zones = categorize_zones(m_pu_avg_fare, m_do_avg_fare, m_avg_fare)[2] # average neighborhoods
type(m_pricier_zones), type(m_cheap_zones), type(m_avg_zones)  # checking to make sure the output is a list of DataFrames

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [56]:
m_pricier_zones.values # lets look into the expensive neighborhoods first

array([['Randalls Island'],
       ['Roosevelt Island'],
       ['Inwood Hill Park'],
       ['Marble Hill'],
       ['Washington Heights North'],
       ['Inwood'],
       ['Highbridge Park'],
       ['Battery Park'],
       ['Washington Heights South'],
       ['Financial District South'],
       ['Hamilton Heights']], dtype=object)

In [57]:
m_cheap_zones.values # now let's look into the cheap neighborhoods

array([['Flatiron'],
       ['Upper West Side South'],
       ['Yorkville West'],
       ['Lincoln Square East'],
       ['Upper East Side North'],
       ['Lenox Hill West'],
       ['Upper East Side South'],
       ['Kips Bay'],
       ['Gramercy'],
       ['Sutton Place/Turtle Bay North']], dtype=object)

In [58]:
m_avg_zones.values # looking at the average neighborhoods

array([['World Trade Center'],
       ['Battery Park City'],
       ['Financial District North'],
       ['Hamilton Heights'],
       ['Seaport'],
       ['Two Bridges/Seward Park'],
       ['Chinatown'],
       ['Central Harlem North'],
       ['Manhattanville'],
       ['TriBeCa/Civic Center'],
       ['East Harlem North'],
       ['Morningside Heights'],
       ['Alphabet City'],
       ['Lower East Side'],
       ['Central Harlem'],
       ['SoHo'],
       ['Stuy Town/Peter Cooper Village'],
       ['Little Italy/NoLiTa'],
       ['West Chelsea/Hudson Yards'],
       ['Meatpacking/West Village West'],
       ['Bloomingdale'],
       ['East Chelsea'],
       ['East Harlem South'],
       ['Hudson Sq'],
       ['Penn Station/Madison Sq West'],
       ['Clinton West'],
       ['Times Sq/Theatre District'],
       ['Greenwich Village South'],
       ['East Village'],
       ['Midtown Center'],
       ['Manhattan Valley'],
       ['Lenox Hill East'],
       ['Central Park'],
       ['Ga

In [59]:
# linking the zones to the original manhattan_cleaned df to create separate dfs for each category
m_expensive_neighborhoods = manhattan_cleaned[manhattan_cleaned['PU_Zone'].isin(m_pricier_zones['pricier_zones']) | manhattan_cleaned['DO_Zone'].isin(m_pricier_zones['pricier_zones'])]
m_cheap_neighborhoods = manhattan_cleaned[manhattan_cleaned['PU_Zone'].isin(m_cheap_zones['cheaper_zones']) | manhattan_cleaned['DO_Zone'].isin(m_cheap_zones['cheaper_zones'])]
m_avg_neighborhoods = manhattan_cleaned[manhattan_cleaned['PU_Zone'].isin(m_avg_zones['average_zones']) | manhattan_cleaned['DO_Zone'].isin(m_avg_zones['average_zones'])]

*Note: Using functions in the data_utils pyscript*
* **'neighborhood_fare_quantiles'**
  * it takes in each categorized zone and returns the 25th, 50th, and 75th percentile of fare amounts
* **'neighborhood_tip_quantiles'**
  * it takes in each categorized zone and returns the 25th, 50th, and 75th percentile of tip amounts

In [60]:
# checking the variability of fare amounts in each neighborhood
m_fare_quantiles = neighborhood_fare_quantiles(m_expensive_neighborhoods, m_avg_neighborhoods, m_cheap_neighborhoods)
m_fare_quantiles

,expensive_neighborhoods,average_neighborhoods,cheap_neighborhoods
Quantile,,,
0.25,17.0,9.3,8.6
0.50,24.7,13.5,12.1
0.75,32.5,19.1,17.7


> Manhattan
* Fares: the biggest shift in range from the 25th to the 75th percentiles is the expensive neighborhoods with a difference of $15.5

In [61]:
# repeating steps above for the other boroughs using functions: Queens
q_canceled_rides = identify_cancelled_rides(queens, ride_id_cols, fee_cols)
queens_cleaned = remove_cancelled_fare_pairs(queens, q_canceled_rides)

q_avg_fare = queens_cleaned['fare_amount'].mean() # overall average fare amount in Queens
q_pu_avg_fare = queens_cleaned.groupby('PU_Zone')['fare_amount'].mean().reset_index(name='avg_fare').sort_values(by='avg_fare', ascending=False) # average fare amount by PU
q_do_avg_fare = queens_cleaned.groupby('DO_Zone')['fare_amount'].mean().reset_index(name='avg_fare').sort_values(by='avg_fare', ascending=False) # average fare amount by DO

q_pricier_zones = categorize_zones(q_pu_avg_fare, q_do_avg_fare, q_avg_fare)[0] # expensive neighborhoods
q_cheap_zones = categorize_zones(q_pu_avg_fare, q_do_avg_fare, q_avg_fare)[1] # cheap neighborhoods
q_avg_zones = categorize_zones(q_pu_avg_fare, q_do_avg_fare, q_avg_fare)[2] # average neighborhoods

# linking the zones to the original queens_cleaned df to create separate dfs for each category
q_expensive_neighborhoods = queens_cleaned[queens_cleaned['PU_Zone'].isin(q_pricier_zones['pricier_zones']) | queens_cleaned['DO_Zone'].isin(q_pricier_zones['pricier_zones'])]
q_cheap_neighborhoods = queens_cleaned[queens_cleaned['PU_Zone'].isin(q_cheap_zones['cheaper_zones']) | queens_cleaned['DO_Zone'].isin(q_cheap_zones['cheaper_zones'])]
q_avg_neighborhoods = queens_cleaned[queens_cleaned['PU_Zone'].isin(q_avg_zones['average_zones']) | queens_cleaned['DO_Zone'].isin(q_avg_zones['average_zones'])]

In [62]:
q_pricier_zones.values

array([['Kew Gardens'],
       ['South Ozone Park'],
       ['Flushing Meadows-Corona Park'],
       ['Breezy Point/Fort Tilden/Riis Beach'],
       ['JFK Airport'],
       ['LaGuardia Airport'],
       ['Rockaway Park'],
       ['Douglaston'],
       ['Jamaica Bay']], dtype=object)

In [63]:
q_cheap_zones.values

array([['Kew Gardens Hills'],
       ['Glendale'],
       ['Jamaica'],
       ['Auburndale'],
       ['Bayside'],
       ['Elmhurst'],
       ['Hollis'],
       ['East Flushing'],
       ['Jamaica Estates'],
       ['Saint Albans'],
       ['Murray Hill-Queens'],
       ['North Corona'],
       ['Saint Michaels Cemetery/Woodside'],
       ['Queensbridge/Ravenswood'],
       ['Woodside'],
       ['Astoria'],
       ['Springfield Gardens North'],
       ['Springfield Gardens South'],
       ['Baisley Park'],
       ['South Jamaica'],
       ['Elmhurst/Maspeth']], dtype=object)

In [64]:
q_avg_zones.values

array([['JFK Airport'],
       ['Jamaica Bay'],
       ['Astoria Park'],
       ['Breezy Point/Fort Tilden/Riis Beach'],
       ['Howard Beach'],
       ['Sunnyside'],
       ['LaGuardia Airport'],
       ['Broad Channel'],
       ['Baisley Park'],
       ['Richmond Hill'],
       ['Hammels/Arverne'],
       ['Springfield Gardens South'],
       ['Forest Park/Highland Park'],
       ['East Elmhurst'],
       ['Glen Oaks'],
       ['Bellerose'],
       ['Far Rockaway'],
       ['Douglaston'],
       ['Ozone Park'],
       ['Long Island City/Hunters Point'],
       ['Maspeth'],
       ['Elmhurst/Maspeth'],
       ['Jackson Heights'],
       ['Rockaway Park'],
       ['Rego Park'],
       ['Laurelton'],
       ['Middle Village'],
       ['Rosedale'],
       ['Long Island City/Queens Plaza'],
       ['Springfield Gardens North'],
       ['Steinway'],
       ['Cambria Heights'],
       ['Bay Terrace/Fort Totten'],
       ['Oakland Gardens'],
       ['Briarwood/Jamaica Hills'],
       ['Quee

In [65]:
# checking the variability of fare amounts in each neighborhood
q_fare_quantiles = neighborhood_fare_quantiles(q_expensive_neighborhoods, q_avg_neighborhoods, q_cheap_neighborhoods)
q_fare_quantiles

,expensive_neighborhoods,average_neighborhoods,cheap_neighborhoods
Quantile,,,
0.25,19.8,14.75,12.80
0.50,33.1,25.00,19.80
0.75,51.3,41.64,30.54


In [66]:
# repeating steps above for the other boroughs: Brooklyn
b_canceled_rides = identify_cancelled_rides(brooklyn, ride_id_cols, fee_cols)
brooklyn_cleaned = remove_cancelled_fare_pairs(brooklyn, b_canceled_rides)

b_avg_fare = brooklyn_cleaned['fare_amount'].mean() # overall average fare amount in Brooklyn
b_pu_avg_fare = brooklyn_cleaned.groupby('PU_Zone')['fare_amount'].mean().reset_index(name='avg_fare').sort_values(by='avg_fare', ascending=False) # average fare amount by PU
b_do_avg_fare = brooklyn_cleaned.groupby('DO_Zone')['fare_amount'].mean().reset_index(name='avg_fare').sort_values(by='avg_fare', ascending=False) # average fare amount by DO

b_pricier_zones = categorize_zones(b_pu_avg_fare, b_do_avg_fare, b_avg_fare)[0] # expensive neighborhoods
b_cheap_zones = categorize_zones(b_pu_avg_fare, b_do_avg_fare, b_avg_fare)[1] # cheap neighborhoods
b_avg_zones = categorize_zones(b_pu_avg_fare, b_do_avg_fare, b_avg_fare)[2] # average neighborhoods

b_expensive_neighborhoods = brooklyn_cleaned[brooklyn_cleaned['PU_Zone'].isin(b_pricier_zones['pricier_zones']) | brooklyn_cleaned['DO_Zone'].isin(b_pricier_zones['pricier_zones'])]
b_cheap_neighborhoods = brooklyn_cleaned[brooklyn_cleaned['PU_Zone'].isin(b_cheap_zones['cheaper_zones']) | brooklyn_cleaned['DO_Zone'].isin(b_cheap_zones['cheaper_zones'])]
b_avg_neighborhoods = brooklyn_cleaned[brooklyn_cleaned['PU_Zone'].isin(b_avg_zones['average_zones']) | brooklyn_cleaned['DO_Zone'].isin(b_avg_zones['average_zones'])]

In [67]:
b_pricier_zones.values

array([['Coney Island'],
       ['Red Hook'],
       ['Marine Park/Floyd Bennett Field'],
       ['Sunset Park West'],
       ['Starrett City'],
       ['Columbia Street'],
       ['South Williamsburg'],
       ['Gravesend'],
       ['Manhattan Beach'],
       ['Bay Ridge'],
       ['Bath Beach'],
       ['Madison']], dtype=object)

In [68]:
b_cheap_zones.values

array([['East Flatbush/Remsen Village'],
       ['East Flatbush/Farragut'],
       ['Bensonhurst West'],
       ['Bensonhurst East'],
       ['Bath Beach'],
       ['Midwood'],
       ['Ocean Parkway South'],
       ['Homecrest'],
       ['Green-Wood Cemetery'],
       ['Williamsburg (South Side)'],
       ['Stuyvesant Heights'],
       ['Crown Heights North'],
       ['Bedford'],
       ['Clinton Hill'],
       ['Carroll Gardens']], dtype=object)

In [69]:
b_avg_zones.values

array([['Cypress Hills'],
       ['Kensington'],
       ['Downtown Brooklyn/MetroTech'],
       ['Greenpoint'],
       ['Sunset Park East'],
       ['Windsor Terrace'],
       ['Bay Ridge'],
       ['Madison'],
       ['DUMBO/Vinegar Hill'],
       ['East Williamsburg'],
       ['Brooklyn Heights'],
       ['Manhattan Beach'],
       ['Brooklyn Navy Yard'],
       ['East New York'],
       ['Prospect Park'],
       ['Boerum Hill'],
       ['Flatbush/Ditmas Park'],
       ['Gravesend'],
       ['Bushwick South'],
       ['Bushwick North'],
       ['Ocean Hill'],
       ['Williamsburg (North Side)'],
       ['Erasmus'],
       ['Fort Greene'],
       ['Park Slope'],
       ['Gowanus'],
       ['Brighton Beach'],
       ['Flatlands'],
       ['Marine Park/Mill Basin'],
       ['Williamsburg (South Side)'],
       ['Canarsie'],
       ['East New York/Pennsylvania Avenue'],
       ['Cobble Hill'],
       ['Prospect Heights'],
       ['Dyker Heights'],
       ['Crown Heights South'],
       

In [70]:
b_fare_quantiles = neighborhood_fare_quantiles(b_expensive_neighborhoods, b_avg_neighborhoods, b_cheap_neighborhoods)
b_fare_quantiles

,expensive_neighborhoods,average_neighborhoods,cheap_neighborhoods
Quantile,,,
0.25,14.160,12.350,12.70
0.50,23.395,19.495,18.56
0.75,33.500,26.460,24.50


In [71]:
# repeating steps above for the other boroughs: Bronx
br_canceled_rides = identify_cancelled_rides(bronx, ride_id_cols, fee_cols)
bronx_cleaned = remove_cancelled_fare_pairs(bronx, br_canceled_rides)

br_avg_fare = bronx_cleaned['fare_amount'].mean() # overall average fare amount in Bronx
br_pu_avg_fare = bronx_cleaned.groupby('PU_Zone')['fare_amount'].mean().reset_index(name='avg_fare').sort_values(by='avg_fare', ascending=False) # average fare amount by PU
br_do_avg_fare = bronx_cleaned.groupby('DO_Zone')['fare_amount'].mean().reset_index(name='avg_fare').sort_values(by='avg_fare', ascending=False) # average fare amount by DO

br_pricier_zones = categorize_zones(br_pu_avg_fare, br_do_avg_fare, br_avg_fare)[0] # expensive neighborhoods
br_cheap_zones = categorize_zones(br_pu_avg_fare, br_do_avg_fare, br_avg_fare)[1] # cheap neighborhoods
br_avg_zones = categorize_zones(br_pu_avg_fare, br_do_avg_fare, br_avg_fare)[2] # average neighborhoods

br_expensive_neighborhoods = bronx_cleaned[bronx_cleaned['PU_Zone'].isin(br_pricier_zones['pricier_zones']) | bronx_cleaned['DO_Zone'].isin(br_pricier_zones['pricier_zones'])]
br_cheap_neighborhoods = bronx_cleaned[bronx_cleaned['PU_Zone'].isin(br_cheap_zones['cheaper_zones']) | bronx_cleaned['DO_Zone'].isin(br_cheap_zones['cheaper_zones'])]
br_avg_neighborhoods = bronx_cleaned[bronx_cleaned['PU_Zone'].isin(br_avg_zones['average_zones']) | bronx_cleaned['DO_Zone'].isin(br_avg_zones['average_zones'])]

In [72]:
br_pricier_zones.values

array([['City Island'],
       ['Riverdale/North Riverdale/Fieldston'],
       ['Country Club'],
       ['Co-Op City'],
       ['Spuyten Duyvil/Kingsbridge'],
       ['Woodlawn/Wakefield']], dtype=object)

In [73]:
br_cheap_zones.values

array([['West Farms/Bronx River'],
       ['Crotona Park East'],
       ['Soundview/Bruckner'],
       ['Belmont'],
       ['Claremont/Bathgate'],
       ['Crotona Park'],
       ['Longwood']], dtype=object)

In [74]:
br_avg_zones.values

array([['Van Cortlandt Park'],
       ['Spuyten Duyvil/Kingsbridge'],
       ['Pelham Bay Park'],
       ['Woodlawn/Wakefield'],
       ['Mott Haven/Port Morris'],
       ['West Concourse'],
       ['Williamsbridge/Olinville'],
       ['Pelham Bay'],
       ['East Tremont'],
       ['Van Nest/Morris Park'],
       ['Schuylerville/Edgewater Park'],
       ['Eastchester'],
       ['Van Cortlandt Village'],
       ['Melrose South'],
       ['East Concourse/Concourse Village'],
       ['Mount Hope'],
       ['Hunts Point'],
       ['Pelham Parkway'],
       ['Highbridge'],
       ['Kingsbridge Heights'],
       ['Allerton/Pelham Gardens'],
       ['Parkchester'],
       ['Norwood'],
       ['Longwood'],
       ['Bronxdale'],
       ['Soundview/Castle Hill'],
       ['Westchester Village/Unionport'],
       ['Bronx Park'],
       ['University Heights/Morris Heights'],
       ['Fordham South'],
       ['Morrisania/Melrose'],
       ['Bedford Park'],
       ['Country Club'],
       ['Belmont'

In [75]:
br_fare_quantiles = neighborhood_fare_quantiles(br_expensive_neighborhoods, br_avg_neighborhoods, br_cheap_neighborhoods)
br_fare_quantiles

,expensive_neighborhoods,average_neighborhoods,cheap_neighborhoods
Quantile,,,
0.25,14.790,11.70,11.17
0.50,22.105,18.00,16.20
0.75,28.975,24.27,20.84


In [76]:
# repeating steps above for the other boroughs: Staten Island
si_canceled_rides = identify_cancelled_rides(staten_island, ride_id_cols, fee_cols)
staten_island_cleaned = remove_cancelled_fare_pairs(staten_island, si_canceled_rides)

si_avg_fare = staten_island_cleaned['fare_amount'].mean() # overall average fare amount in Staten Island
si_pu_avg_fare = staten_island_cleaned.groupby('PU_Zone')['fare_amount'].mean().reset_index(name='avg_fare').sort_values(by='avg_fare', ascending=False) # average fare amount by PU
si_do_avg_fare = staten_island_cleaned.groupby('DO_Zone')['fare_amount'].mean().reset_index(name='avg_fare').sort_values(by='avg_fare', ascending=False) # average fare amount by DO

si_pricier_zones = categorize_zones(si_pu_avg_fare, si_do_avg_fare, si_avg_fare)[0] # expensive neighborhoods
si_cheap_zones = categorize_zones(si_pu_avg_fare, si_do_avg_fare, si_avg_fare)[1] # cheap neighborhoods
si_avg_zones = categorize_zones(si_pu_avg_fare, si_do_avg_fare, si_avg_fare)[2] # average neighborhoods

si_expensive_neighborhoods = staten_island_cleaned[staten_island_cleaned['PU_Zone'].isin(si_pricier_zones['pricier_zones']) | staten_island_cleaned['DO_Zone'].isin(si_pricier_zones['pricier_zones'])]
si_cheap_neighborhoods = staten_island_cleaned[staten_island_cleaned['PU_Zone'].isin(si_cheap_zones['cheaper_zones']) | staten_island_cleaned['DO_Zone'].isin(si_cheap_zones['cheaper_zones'])]
si_avg_neighborhoods = staten_island_cleaned[staten_island_cleaned['PU_Zone'].isin(si_avg_zones['average_zones']) | staten_island_cleaned['DO_Zone'].isin(si_avg_zones['average_zones'])]

In [77]:
si_pricier_zones.values

array([['Arden Heights'],
       ['Charleston/Tottenville'],
       ['Oakwood']], dtype=object)

In [78]:
si_cheap_zones.values

array([['Arrochar/Fort Wadsworth'],
       ['Westerleigh'],
       ['Port Richmond']], dtype=object)

In [79]:
si_avg_zones.values

array([['Heartland Village/Todt Hill'],
       ['Rossville/Woodrow'],
       ['Saint George/New Brighton'],
       ['West Brighton'],
       ['Oakwood'],
       ['South Beach/Dongan Hills'],
       ['Bloomfield/Emerson Hill'],
       ['Mariners Harbor'],
       ['New Dorp/Midland Beach'],
       ['Port Richmond'],
       ['Grymes Hill/Clifton'],
       ['Stapleton'],
       ["Eltingville/Annadale/Prince's Bay"],
       ['Arrochar/Fort Wadsworth']], dtype=object)

In [80]:
si_fare_quantiles = neighborhood_fare_quantiles(si_expensive_neighborhoods, si_avg_neighborhoods, si_cheap_neighborhoods)
si_fare_quantiles

,expensive_neighborhoods,average_neighborhoods,cheap_neighborhoods
Quantile,,,
0.25,12.685,8.2600,4.64
0.50,16.560,17.7900,10.65
0.75,135.000,25.4925,18.27


#### **Key Findings Pt. 1:**

**Fare Amounts**:
1. Fare prices are based off distance:
  * Overall averages show that Queens & Staten Island have pricier fares, wile Manhattan has the cheapest fares
  * Queens & Staten Island having pricier fares is largely due to its geographic characteristics, which makes sense because these are generally larger and less dense leading to longer trips
  * While Manhattan which is more compact and have shorter trips

2. Neighborhood Segmentation:
  * Expensive neighborhoods: 
    * Queens stand out with a much higher 75th percentile at $51.30, which reflects longer trips, including airports (recall, a good chunk of Queens’ trips are airport related)
    * Manhattan’s 75th percentile at $32.68 shows that in a generally cheaper borough, there exists very expensive neighborhoods with costlier fares
    * Staten Island has an extremely high 75th percentile at $135 in comparison to its lower percentiles, showing that there exists rare occurrences of expensive trips
      * Which makes sense considering taxi trips to Staten Island are the least popular, generally the rides could be extremely long depending on the area
    * Even in Bronx’s most expensive neighborhoods, it doesn’t rarely reaches the highs seen in other boroughs

3. Cheaper neighborhoods: 
  * Manhattan has the lowest 50th percentile of $8.60, confirming that shorter, cheaper rides are common 
  * Queens and Staten Island show the most variability in their fares, most likely due to having many cheaper rides, with occasion of very expensive rides which pulls their averages up
  * Bronx’s fare seem to be on the lower side, and unlike Manhattan, it’s more consistent with less variability 
    * This could suggests more localized trips and lesser expensive outlier rides compared to Queens or Staten Island

4. Average neighborhoods:
  * Brooklyn seem to be the most representative of NYC’s overall average fare amount with its percentiles closest to the overall NYC fare average of $20.30
    * This suggests that Brooklyn’s fares could be the most representative of NYC as a whole
  * Manhattan’s average neighborhoods still remain cheaper than the overall NYC average


> Now that we looked over the different categories of whether the neighborhood is: pricier, average, or cheaper
* Lets investigate the tips columns to the corresponding fare amounts for any potential input errors or outliers we need to take care of
  * Tip to fare ratio
  * **Tip to total amount ratio**
    * *exclude other charges for the total* --> tip/(tip + total)


In [81]:
# variability of tip amounts, definitely seems to have extreme outliers
manhattan_cleaned['tip_amount'].quantile([0, 0.25, 0.5, 0.75, 1])  

0.00      0.00
0.25      0.00
0.50      2.38
0.75      3.79
1.00    443.21
Name: tip_amount, dtype: float64

In [82]:
# fare amounts over the 75th percentile of tip amounts, sorted by the most fare amount
# the goal is to visualize what a good threshold for tip amounts should be
tip_75th_perc = manhattan_cleaned[manhattan_cleaned['tip_amount'] > manhattan_cleaned['tip_amount'].quantile(0.75)].sort_values(by=['fare_amount', 'tip_amount'], ascending=False).reset_index(drop=True).head(5)  
tip_75th_perc[['fare_amount', 'tip_amount', 'total_amount']].head() 

,fare_amount,tip_amount,total_amount
0,400.0,92.00,496.25
1,400.0,80.35,482.10
2,325.0,65.85,395.10
3,300.0,60.95,365.70
4,300.0,60.85,365.10


> This is showing even the highest tips for the highests fare within the 75th percentile don't exceed $100, meaning most values much higher than $100 are likely to be outliers

In [83]:
# picking a percentile that best represents that number: 0.99999 --> $99
manhattan_cleaned['tip_amount'].quantile([0.9999, 0.99999, 0.999999, 1])
tip_quant = manhattan_cleaned['tip_amount'].quantile(0.99999)

In [84]:
# displaying disproportionately high tip amounts compared to fare amounts
tip_selected_perc = manhattan_cleaned[manhattan_cleaned['tip_amount'] > manhattan_cleaned['tip_amount'].quantile(0.99999)].sort_values(by=['tip_amount','fare_amount'], ascending=False).reset_index(drop=True)
tip_selected_perc[['fare_amount', 'tip_amount', 'total_amount']].head() 

,fare_amount,tip_amount,total_amount
0,7.20,443.21,457.66
1,24.70,250.00,281.95
2,9.30,222.00,238.55
3,14.90,200.00,219.65
4,0.01,174.44,175.45


* We have to keep the df sorted by tip_amount, fare_amount DESC
* Also make sure that tip amount > fare amount 


In [85]:
# removing rides with fare_amount < 1, since they can skew the data due to atypical fare amounts
manhattan_cleaned = manhattan_cleaned[manhattan_cleaned['fare_amount'] >= 1].copy()

# find the tip to fare and tip to total ratios for the rest of the columns
manhattan_cleaned['tip_fare_ratio'] = (manhattan_cleaned['tip_amount'] / manhattan_cleaned['fare_amount']) * 100
manhattan_cleaned['tip_fare_ratio'] = manhattan_cleaned['tip_fare_ratio'].round(2)
manhattan_cleaned['tip_total_ratio'] = (manhattan_cleaned['tip_amount'] / (manhattan_cleaned['tip_amount'] + manhattan_cleaned['fare_amount'])) * 100
manhattan_cleaned['tip_total_ratio'] = manhattan_cleaned['tip_total_ratio'].round(2)

# rearranging the columns to have the monetary columns at the end for better readability
monetary_cols = ['fare_amount', 'tip_amount', 'tip_fare_ratio', 'tip_total_ratio','total_amount', 'extra', 'mta_tax', 'improvement_surcharge', 'tolls_amount', 'congestion_surcharge', 'cbd_congestion_fee', 'Airport_fee']
manhattan_tips = manhattan_cleaned[monetary_cols + [c for c in manhattan_cleaned.columns if c not in monetary_cols]].sort_values(by=['tip_amount','fare_amount'], ascending=False).copy()
manhattan_tips[manhattan_tips['tip_amount'] > manhattan_tips['fare_amount']].head() # rides where tip amount > fare amount, sorted by tip amount & fare amount descending

,fare_amount,tip_amount,tip_fare_ratio,tip_total_ratio,total_amount,extra,mta_tax,improvement_surcharge,tolls_amount,congestion_surcharge,cbd_congestion_fee,Airport_fee,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
2587129,7.2,443.21,6155.69,98.40,457.66,2.50,0.5,1.0,0.0,2.5,0.75,0.0,2025-05-30 19:41:42,2025-05-30 19:47:36,5.900000,0.65,1.0,UN/Turtle Bay South,Yellow Zone,Murray Hill,Yellow Zone,2,1.0,233,170,1,Manhattan,Manhattan,"Curb Mobility, LLC",Credit card,Standard
36053,24.7,250.00,1012.15,91.01,281.95,5.75,0.5,1.0,0.0,2.5,0.75,0.0,2025-05-01 19:02:03,2025-05-01 19:29:30,27.450000,3.10,1.0,Midtown North,Yellow Zone,Meatpacking/West Village West,Yellow Zone,1,1.0,163,158,1,Manhattan,Manhattan,"Creative Mobile Technologies, LLC",Credit card,Standard
1026468,9.3,222.00,2387.10,95.98,238.55,2.50,0.5,1.0,0.0,2.5,0.75,0.0,2025-05-12 16:00:46,2025-05-12 16:09:34,8.800000,0.71,1.0,Lenox Hill West,Yellow Zone,Midtown East,Yellow Zone,2,1.0,141,162,1,Manhattan,Manhattan,"Curb Mobility, LLC",Credit card,Standard
1593599,14.9,200.00,1342.28,93.07,219.65,0.00,0.5,1.0,0.0,2.5,0.75,0.0,2025-05-18 16:04:08,2025-05-18 16:18:33,14.416667,2.17,1.0,Times Sq/Theatre District,Yellow Zone,Upper West Side South,Yellow Zone,2,1.0,230,239,1,Manhattan,Manhattan,"Curb Mobility, LLC",Credit card,Standard
2235800,14.9,150.00,1006.71,90.96,169.65,0.00,0.5,1.0,0.0,2.5,0.75,0.0,2025-05-26 19:39:44,2025-05-26 19:53:07,13.383333,2.37,1.0,Midtown East,Yellow Zone,Lenox Hill East,Yellow Zone,2,1.0,162,140,1,Manhattan,Manhattan,"Curb Mobility, LLC",Credit card,Standard


In [86]:
# creating a mask to filter out the outliers in tip amounts
outlier_mask = ((manhattan_tips['tip_amount'] > manhattan_tips['fare_amount']) & (manhattan_tips['tip_amount'] > tip_quant))
manhattan_tips = manhattan_tips[~outlier_mask]  # checking the statistics of the non-outlier tips

> after filtering out tip amounts over 100, further filter:
* a good place to start is by looking at certain ratios --> using as a threshold
  * tip to fare
  * tip to total


In [87]:
manhattan_tips['tip_total_ratio'].quantile([0.995, 0.999, 0.9999, 1])  # checking the quantiles of tip to total ratio

0.9950    37.27
0.9990    51.95
0.9999    83.33
1.0000    97.06
Name: tip_total_ratio, dtype: float64

In [88]:
manhattan_tips['tip_fare_ratio'].quantile([0.995, 0.999, 0.9999, 1])  # checking the quantiles of tip to total ratio

0.9950      59.41
0.9990     108.11
0.9999     500.00
1.0000    3300.00
Name: tip_fare_ratio, dtype: float64

In [89]:
# set a threshold for each ratio: extremely generous thresholds to filter out the extreme outliers
tip_total_ratio_threshold = manhattan_tips['tip_total_ratio'].quantile(0.999) 
tip_fare_ratio_threshold = manhattan_tips['tip_fare_ratio'].quantile(0.999)

# further filtering with ratios
manhattan_tips = manhattan_tips[(manhattan_tips['tip_fare_ratio'] < tip_fare_ratio_threshold) & (manhattan_tips['tip_total_ratio'] < tip_total_ratio_threshold)]
manhattan_tips.head()  # checking the final dataframe after filtering out the extreme outliers

,fare_amount,tip_amount,tip_fare_ratio,tip_total_ratio,total_amount,extra,mta_tax,improvement_surcharge,tolls_amount,congestion_surcharge,cbd_congestion_fee,Airport_fee,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
2442524,157.0,99.99,63.69,38.91,261.74,0.00,0.5,1.0,0.0,2.5,0.75,0.0,2025-05-29 10:46:00,2025-05-29 13:34:37,168.616667,23.14,1.0,Gramercy,Yellow Zone,Upper West Side South,Yellow Zone,2,1.0,107,239,1,Manhattan,Manhattan,"Curb Mobility, LLC",Credit card,Standard
710306,400.0,92.00,23.00,18.70,496.25,0.00,0.0,1.0,0.0,2.5,0.75,0.0,2025-05-09 00:23:35,2025-05-09 00:23:45,0.166667,0.00,4.0,Central Park,Yellow Zone,Central Park,Yellow Zone,2,5.0,43,43,1,Manhattan,Manhattan,"Curb Mobility, LLC",Credit card,Negotiated fare
27979,84.3,86.21,102.27,50.56,174.76,0.00,0.0,1.0,0.0,2.5,0.75,0.0,2025-05-01 18:39:33,2025-05-01 18:39:39,0.100000,0.00,4.0,Clinton East,Yellow Zone,Clinton East,Yellow Zone,2,5.0,48,48,1,Manhattan,Manhattan,"Curb Mobility, LLC",Credit card,Negotiated fare
342073,400.0,80.35,20.09,16.73,482.10,0.75,0.0,1.0,0.0,0.0,0.75,0.0,2025-05-05 08:54:51,2025-05-05 12:12:24,197.550000,13.70,1.0,Lenox Hill West,Yellow Zone,Flatiron,Yellow Zone,1,5.0,141,90,1,Manhattan,Manhattan,"Creative Mobile Technologies, LLC",Credit card,Negotiated fare
914025,140.0,70.00,50.00,33.33,211.75,0.75,0.0,1.0,0.0,0.0,0.75,0.0,2025-05-11 04:02:05,2025-05-11 04:02:29,0.400000,0.00,1.0,East Village,Yellow Zone,East Village,Yellow Zone,1,5.0,79,79,1,Manhattan,Manhattan,"Creative Mobile Technologies, LLC",Credit card,Negotiated fare


*Note: saved the entire cleaning process above into a functions in the data_utils pyscript*
* **'borough_tip_cleaned'**
  * cleans the borough dataframe for tip analysis
    - calculates tip to fare and tip to total ratios
    - removes outliers based on tip amounts
    - filters out extreme outliers based on tip to fare and tip to total ratios
    - returns a cleaned dataframe with rearranged columns for readability

In [90]:
# repeating the same process for Queens
queens_tips = borough_tip_cleaned(queens_cleaned)
queens_tips.head()  

,fare_amount,tip_amount,tip_fare_ratio,tip_total_ratio,total_amount,extra,mta_tax,improvement_surcharge,tolls_amount,congestion_surcharge,cbd_congestion_fee,Airport_fee,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
46946,525.00,157.80,30.06,23.11,683.80,0.0,0.0,1.0,0.0,0.0,0.0,0.00,2025-05-20 16:22:06,2025-05-20 16:26:20,4.233333,2.10,1.0,JFK Airport,Airports,JFK Airport,Airports,1,5.0,132,132,1,Queens,Queens,"Creative Mobile Technologies, LLC",Credit card,Negotiated fare
14494,559.00,112.00,20.04,16.69,672.00,0.0,0.0,1.0,0.0,0.0,0.0,0.00,2025-05-07 11:28:57,2025-05-07 11:29:11,0.233333,0.00,2.0,Springfield Gardens North,Boro Zone,Springfield Gardens North,Boro Zone,2,5.0,218,218,1,Queens,Queens,"Curb Mobility, LLC",Credit card,Negotiated fare
41478,89.99,99.99,111.11,52.63,197.73,5.0,0.0,1.0,0.0,0.0,0.0,1.75,2025-05-18 15:01:17,2025-05-18 15:01:24,0.116667,0.00,2.0,LaGuardia Airport,Airports,LaGuardia Airport,Airports,2,5.0,138,138,1,Queens,Queens,"Curb Mobility, LLC",Credit card,Negotiated fare
20823,145.50,99.00,68.04,40.49,248.25,1.0,0.0,1.0,0.0,0.0,0.0,1.75,2025-05-10 05:30:42,2025-05-10 06:27:02,56.333333,32.84,1.0,JFK Airport,Airports,JFK Airport,Airports,2,3.0,132,132,1,Queens,Queens,"Curb Mobility, LLC",Credit card,Newark
57331,122.00,99.00,81.15,44.80,224.25,0.0,0.5,1.0,0.0,0.0,0.0,1.75,2025-05-25 06:27:01,2025-05-25 06:57:16,30.250000,20.27,1.0,JFK Airport,Airports,JFK Airport,Airports,2,4.0,132,132,1,Queens,Queens,"Curb Mobility, LLC",Credit card,Nassau or Westchester


In [91]:
# repeating the same process for Brooklyn
brooklyn_tips = borough_tip_cleaned(brooklyn_cleaned)
brooklyn_tips.head()

,fare_amount,tip_amount,tip_fare_ratio,tip_total_ratio,total_amount,extra,mta_tax,improvement_surcharge,tolls_amount,congestion_surcharge,cbd_congestion_fee,Airport_fee,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
2097,202.5,92.75,45.80,31.41,300.0,3.25,0.5,1.0,0.0,2.5,0.75,0.0,2025-05-04 10:44:30,2025-05-04 15:13:12,268.700000,13.80,1.0,Greenpoint,Boro Zone,Greenpoint,Boro Zone,1,1.0,112,112,1,Brooklyn,Brooklyn,"Creative Mobile Technologies, LLC",Credit card,Standard
10471,450.0,49.00,10.89,9.82,500.0,0.00,0.0,1.0,0.0,0.0,0.00,0.0,2025-05-17 05:37:54,2025-05-17 05:38:52,0.966667,0.00,1.0,Kensington,Boro Zone,Kensington,Boro Zone,2,5.0,133,133,1,Brooklyn,Brooklyn,"Curb Mobility, LLC",Credit card,Negotiated fare
13510,450.0,49.00,10.89,9.82,500.0,0.00,0.0,1.0,0.0,0.0,0.00,0.0,2025-05-22 03:46:36,2025-05-22 03:47:11,0.583333,0.00,1.0,Kensington,Boro Zone,Kensington,Boro Zone,2,5.0,133,133,1,Brooklyn,Brooklyn,"Curb Mobility, LLC",Credit card,Negotiated fare
14949,450.0,49.00,10.89,9.82,500.0,0.00,0.0,1.0,0.0,0.0,0.00,0.0,2025-05-24 05:33:37,2025-05-24 05:41:31,7.900000,1.12,1.0,Kensington,Boro Zone,Kensington,Boro Zone,2,5.0,133,133,1,Brooklyn,Brooklyn,"Curb Mobility, LLC",Credit card,Negotiated fare
2295,220.0,44.20,20.09,16.73,265.2,0.00,0.0,1.0,0.0,0.0,0.00,0.0,2025-05-04 16:40:56,2025-05-04 16:42:14,1.300000,0.27,4.0,Downtown Brooklyn/MetroTech,Boro Zone,Downtown Brooklyn/MetroTech,Boro Zone,2,5.0,65,65,1,Brooklyn,Brooklyn,"Curb Mobility, LLC",Credit card,Negotiated fare


In [92]:
# repeating the same process for Bronx
bronx_tips = borough_tip_cleaned(bronx_cleaned)
bronx_tips.head()

,fare_amount,tip_amount,tip_fare_ratio,tip_total_ratio,total_amount,extra,mta_tax,improvement_surcharge,tolls_amount,congestion_surcharge,cbd_congestion_fee,Airport_fee,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
1050,450.00,45.00,10.00,9.09,496.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2025-05-18 19:50:02,2025-05-18 19:50:27,0.416667,0.0,1.0,Van Nest/Morris Park,Boro Zone,Van Nest/Morris Park,Boro Zone,2,5.0,242,242,1,Bronx,Bronx,"Curb Mobility, LLC",Credit card,Negotiated fare
920,100.00,30.45,30.45,23.34,131.95,0.0,0.5,1.0,0.0,0.0,0.0,0.0,2025-05-16 22:24:42,2025-05-16 22:24:42,0.000000,0.0,1.0,Co-Op City,Boro Zone,Co-Op City,Boro Zone,7,5.0,51,51,1,Bronx,Bronx,Helix,Credit card,Negotiated fare
493,122.79,24.76,20.16,16.78,148.55,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2025-05-08 20:30:16,2025-05-08 20:30:26,0.166667,0.0,1.0,Riverdale/North Riverdale/Fieldston,Boro Zone,Riverdale/North Riverdale/Fieldston,Boro Zone,2,5.0,200,200,1,Bronx,Bronx,"Curb Mobility, LLC",Credit card,Negotiated fare
1119,74.40,23.05,30.98,23.65,99.95,1.0,0.5,1.0,0.0,0.0,0.0,0.0,2025-05-20 02:04:45,2025-05-20 03:11:47,67.033333,12.6,1.0,Mount Hope,Boro Zone,Morrisania/Melrose,Boro Zone,1,1.0,169,167,1,Bronx,Bronx,"Creative Mobile Technologies, LLC",Credit card,Standard
1755,74.94,22.78,30.40,23.31,98.72,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2025-05-29 18:24:32,2025-05-29 18:24:37,0.083333,0.0,1.0,West Farms/Bronx River,Boro Zone,West Farms/Bronx River,Boro Zone,2,5.0,248,248,1,Bronx,Bronx,"Curb Mobility, LLC",Credit card,Negotiated fare


In [93]:
# repeating the same process for Staten Island
staten_island_tips = borough_tip_cleaned(staten_island_cleaned)
staten_island_tips.head()

,fare_amount,tip_amount,tip_fare_ratio,tip_total_ratio,total_amount,extra,mta_tax,improvement_surcharge,tolls_amount,congestion_surcharge,cbd_congestion_fee,Airport_fee,PU_datetime,DO_datetime,duration_mins,trip_distance,passenger_count,PU_Zone,PU_service_zone,DO_Zone,DO_service_zone,VendorID,RatecodeID,PULocationID,DOLocationID,payment_type,PU_Borough,DO_Borough,vendor_name,payment_method,Rate
30,145.89,29.38,20.14,16.76,176.27,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2025-05-19 09:33:27,2025-05-19 09:41:47,8.333333,1.76,4.0,South Beach/Dongan Hills,Boro Zone,New Dorp/Midland Beach,Boro Zone,2,5.0,214,172,1,Staten Island,Staten Island,"Curb Mobility, LLC",Credit card,Negotiated fare
33,140.00,28.20,20.14,16.77,169.20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2025-05-24 20:23:39,2025-05-24 20:23:45,0.100000,0.00,2.0,Oakwood,Boro Zone,Oakwood,Boro Zone,2,5.0,176,176,1,Staten Island,Staten Island,"Curb Mobility, LLC",Credit card,Negotiated fare
18,130.00,20.00,15.38,13.33,151.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2025-05-09 22:15:07,2025-05-09 22:15:13,0.100000,0.00,1.0,Charleston/Tottenville,Boro Zone,Charleston/Tottenville,Boro Zone,2,5.0,44,44,1,Staten Island,Staten Island,"Curb Mobility, LLC",Credit card,Negotiated fare
12,40.00,12.30,30.75,23.52,53.30,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2025-05-06 21:53:27,2025-05-06 21:53:32,0.083333,0.00,1.0,Grymes Hill/Clifton,Boro Zone,Grymes Hill/Clifton,Boro Zone,2,5.0,115,115,1,Staten Island,Staten Island,"Curb Mobility, LLC",Credit card,Negotiated fare
10,136.00,10.00,7.35,6.85,147.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2025-05-05 14:14:52,2025-05-05 14:15:50,0.966667,0.43,1.0,Heartland Village/Todt Hill,Boro Zone,Heartland Village/Todt Hill,Boro Zone,2,5.0,118,118,1,Staten Island,Staten Island,"Curb Mobility, LLC",Credit card,Negotiated fare


In [94]:
# staten island's records decreased significantly
staten_island_tips.shape[0] 

106

> Now we have to match up the values specific to the borough's neighborhoods to the newly cleaned tips df

*Note*:
I'll be using **'_tips'** for all the analysis with tipping behavior from here on out for neighborhood analysis

In [95]:
# Manhattan
# since the original df didnt have a unique ride identifier, we will create one based on these columns that uniquely identify a ride
# recall that the 'ride_id_cols' were used to identify cancelled rides, so we can use the same columns to create a unique ride identifier
manhattan_tips = manhattan_tips.copy()
m_expensive_neighborhoods = m_expensive_neighborhoods.copy()

# saving the keys to a new column for easier filtering later
manhattan_tips['ride_key'] = manhattan_tips[ride_id_cols].astype(str).agg('_'.join, axis=1) 
m_expensive_neighborhoods['ride_key'] = m_expensive_neighborhoods[ride_id_cols].astype(str).agg('_'.join, axis=1)

# filter manhattan_tips for records in m_expensive_neighborhoods
m_exp_tips = manhattan_tips[manhattan_tips['ride_key'].isin(m_expensive_neighborhoods['ride_key'])]

*Note: saved the entire matching process above into a functions in the data_utils pyscript*
* **'match_tip_neighborhoods'**
  * match tips with neighborhoods based on ride_id_cols & returns a dataframe with matched tips and its respective neighborhood
    - ride_id_cols serve as a unique ride identification
    - filters borough_tips for records in borough_neighborhoods

In [96]:
# doing the rest of the neighborhoods for each borough
m_avg_tips = match_tip_neighborhoods(manhattan_tips, m_avg_neighborhoods, ride_id_cols)
m_cheap_tips = match_tip_neighborhoods(manhattan_tips, m_cheap_neighborhoods, ride_id_cols)

# Queens
q_exp_tips = match_tip_neighborhoods(queens_tips, q_expensive_neighborhoods, ride_id_cols)
q_avg_tips = match_tip_neighborhoods(queens_tips, q_avg_neighborhoods, ride_id_cols)
q_cheap_tips = match_tip_neighborhoods(queens_tips, q_cheap_neighborhoods, ride_id_cols)

# Brooklyn
b_exp_tips = match_tip_neighborhoods(brooklyn_tips, b_expensive_neighborhoods, ride_id_cols)
b_avg_tips = match_tip_neighborhoods(brooklyn_tips, b_avg_neighborhoods, ride_id_cols)
b_cheap_tips = match_tip_neighborhoods(brooklyn_tips, b_cheap_neighborhoods, ride_id_cols)

# Bronx
br_exp_tips = match_tip_neighborhoods(bronx_tips, br_expensive_neighborhoods, ride_id_cols)
br_avg_tips = match_tip_neighborhoods(bronx_tips, br_avg_neighborhoods, ride_id_cols)
br_cheap_tips = match_tip_neighborhoods(bronx_tips, br_cheap_neighborhoods, ride_id_cols)

# Staten Island
si_exp_tips = match_tip_neighborhoods(staten_island_tips, si_expensive_neighborhoods, ride_id_cols)
si_avg_tips = match_tip_neighborhoods(staten_island_tips, si_avg_neighborhoods, ride_id_cols)
si_cheap_tips = match_tip_neighborhoods(staten_island_tips, si_cheap_neighborhoods, ride_id_cols)

> As a whole, which areas have the highest/lowest average tips?
* Looking into the variability of tip amounts in each boroughs' neighborhood 
* Using categorized zones to analyze tipping behavior in Manhattan

In [97]:
# average tip amount in each borough
pd.DataFrame({
    'Manhattan': manhattan_tips['tip_amount'].mean(),
    'Queens': queens_tips['tip_amount'].mean(),
    'Brooklyn': brooklyn_tips['tip_amount'].mean(),
    'Bronx': bronx_tips['tip_amount'].mean(),
    'Staten Island': staten_island_tips['tip_amount'].mean()
}, index=[0])

,Manhattan,Queens,Brooklyn,Bronx,Staten Island
0,2.404988,2.825446,0.311359,0.068469,1.203208


> Seems like Manhattan and Queens has the highest average tips in their respective borough 

> Lets look at tipping behaviors according to each borough's neighborhoods to see if there are any trends:

In [98]:
# Manhattan
# max tips grouped by PU_Borough & DO_Borough
manhattan_tips.groupby(['PU_Zone', 'DO_Zone'])[['fare_amount','tip_amount']].max().sort_values(by='tip_amount', ascending=False).head()  

,,fare_amount,tip_amount
PU_Zone,DO_Zone,,
Gramercy,Upper West Side South,157.0,99.99
Central Park,Central Park,500.0,92.00
Clinton East,Clinton East,237.0,86.21
Lenox Hill West,Flatiron,400.0,80.35
East Village,East Village,297.0,70.00


In [99]:
# Manhattan: neighborhoods with the lowest average tips
m_tip_pu = manhattan_tips.groupby('PU_Zone')['tip_amount'].agg(avg_tip_PU='mean', max_tip_PU='max', min_tip_PU='min').reset_index() # avg, min, max tip by pickup zone
m_tip_do = manhattan_tips.groupby('DO_Zone')['tip_amount'].agg(avg_tip_DO='mean', max_tip_DO='max', min_tip_DO='min').reset_index() # avg, min, max tip by dropoff zone
m_tip_zones = pd.merge(m_tip_pu, m_tip_do, left_on='PU_Zone', right_on='DO_Zone', how='inner').sort_values(by=['avg_tip_PU','avg_tip_DO'], ascending=True) # merge on zones
m_tip_zones[['PU_Zone', 'avg_tip_PU', 'avg_tip_DO', 'max_tip_PU', 'max_tip_DO', 'min_tip_PU', 'min_tip_DO']].head()

,PU_Zone,avg_tip_PU,avg_tip_DO,max_tip_PU,max_tip_DO,min_tip_PU,min_tip_DO
22,Highbridge Park,0.149744,0.970758,8.58,12.97,0.0,0.0
24,Inwood,0.174815,1.810329,17.80,22.70,0.0,0.0
35,Marble Hill,0.184706,1.903147,15.20,15.20,0.0,0.0
5,Central Harlem North,0.334307,1.535337,26.40,26.40,0.0,0.0
59,Washington Heights North,0.397747,2.289058,24.20,30.00,0.0,0.0


In [100]:
# Manhattan: neighborhoods with the highest average tips
m_tip_zones[['PU_Zone', 'avg_tip_PU', 'avg_tip_DO', 'max_tip_PU', 'max_tip_DO', 'min_tip_PU', 'min_tip_DO']].tail()

,PU_Zone,avg_tip_PU,avg_tip_DO,max_tip_PU,max_tip_DO,min_tip_PU,min_tip_DO
43,Penn Station/Madison Sq West,2.887477,2.307089,60.95,60.95,0.0,0.0
63,World Trade Center,2.911478,2.708859,55.00,55.00,0.0,0.0
2,Battery Park City,3.010589,3.069496,30.68,45.80,0.0,0.0
1,Battery Park,3.034722,2.780498,20.00,20.00,0.0,0.0
44,Randalls Island,7.085728,4.521337,30.00,25.00,0.0,0.0


> Repeating the same steps as above for the rest of the boroughs:

In [101]:
# Queens
# max tips grouped by PU_Borough & DO_Borough
queens_tips.groupby(['PU_Zone', 'DO_Zone'])[['fare_amount','tip_amount']].max().sort_values(by='tip_amount', ascending=False).head()  

,,fare_amount,tip_amount
PU_Zone,DO_Zone,,
JFK Airport,JFK Airport,900.0,157.80
Springfield Gardens North,Springfield Gardens North,559.0,112.00
LaGuardia Airport,LaGuardia Airport,650.0,99.99
JFK Airport,Long Island City/Hunters Point,121.0,98.90
South Ozone Park,JFK Airport,165.0,98.00


In [102]:
# neighborhoods with the lowest average tips
q_tip_pu = queens_tips.groupby('PU_Zone')['tip_amount'].agg(avg_tip_PU='mean', max_tip_PU='max', min_tip_PU='min').reset_index() # avg, min, max tip by pickup zone
q_tip_do = queens_tips.groupby('DO_Zone')['tip_amount'].agg(avg_tip_DO='mean', max_tip_DO='max', min_tip_DO='min').reset_index() # avg, min, max tip by dropoff zone
q_tip_zones = pd.merge(q_tip_pu, q_tip_do, left_on='PU_Zone', right_on='DO_Zone', how='inner').sort_values(by=['avg_tip_PU','avg_tip_DO'], ascending=True) # merge on zones
q_tip_zones[['PU_Zone', 'avg_tip_PU', 'avg_tip_DO', 'max_tip_PU', 'max_tip_DO', 'min_tip_PU', 'min_tip_DO']].head()

,PU_Zone,avg_tip_PU,avg_tip_DO,max_tip_PU,max_tip_DO,min_tip_PU,min_tip_DO
9,Broad Channel,0.000000,3.147143,0.00,24.45,0.0,0.0
33,Jamaica Bay,0.000000,5.140000,0.00,20.00,0.0,0.0
7,Breezy Point/Fort Tilden/Riis Beach,0.000000,7.229855,0.00,29.30,0.0,0.0
26,Hammels/Arverne,0.003861,1.926644,1.00,68.95,0.0,0.0
28,Hollis,0.025528,0.965873,5.75,10.94,0.0,0.0


In [103]:
# neighborhoods with the highest average tips
q_tip_zones[['PU_Zone', 'avg_tip_PU', 'avg_tip_DO', 'max_tip_PU', 'max_tip_DO', 'min_tip_PU', 'min_tip_DO']].tail()

,PU_Zone,avg_tip_PU,avg_tip_DO,max_tip_PU,max_tip_DO,min_tip_PU,min_tip_DO
35,Kew Gardens,4.795108,3.049273,93.27,93.27,0.0,0.0
37,LaGuardia Airport,4.899783,7.049847,99.99,99.99,0.0,0.0
20,Flushing Meadows-Corona Park,4.904807,0.809189,40.20,60.00,0.0,0.0
59,South Ozone Park,5.189325,3.038866,98.00,84.43,0.0,0.0
1,Astoria Park,5.707333,2.185484,19.98,17.41,0.0,0.0


In [104]:
# Brooklyn
brooklyn_tips.groupby(['PU_Zone', 'DO_Zone'])[['fare_amount','tip_amount']].max().sort_values(by='tip_amount', ascending=False).head()  

,,fare_amount,tip_amount
PU_Zone,DO_Zone,,
Greenpoint,Greenpoint,250.0,92.75
Kensington,Kensington,450.0,49.00
Downtown Brooklyn/MetroTech,Downtown Brooklyn/MetroTech,220.0,44.20
Boerum Hill,Boerum Hill,233.0,40.20
Bay Ridge,Red Hook,200.0,40.20


In [105]:
b_tip_pu = brooklyn_tips.groupby('PU_Zone')['tip_amount'].agg(avg_tip_PU='mean', max_tip_PU='max', min_tip_PU='min').reset_index() # avg, min, max tip by pickup zone
b_tip_do = brooklyn_tips.groupby('DO_Zone')['tip_amount'].agg(avg_tip_DO='mean', max_tip_DO='max', min_tip_DO='min').reset_index() # avg, min, max tip by dropoff zone
b_tip_zones = pd.merge(b_tip_pu, b_tip_do, left_on='PU_Zone', right_on='DO_Zone', how='inner').sort_values(by=['avg_tip_PU','avg_tip_DO'], ascending=True) # merge on zones
b_tip_zones[['PU_Zone', 'avg_tip_PU', 'avg_tip_DO', 'max_tip_PU', 'max_tip_DO', 'min_tip_PU', 'min_tip_DO']].head()

,PU_Zone,avg_tip_PU,avg_tip_DO,max_tip_PU,max_tip_DO,min_tip_PU,min_tip_DO
54,Starrett City,0.012564,0.000000,5.88,0.00,0.0,0.0
6,Borough Park,0.013850,0.049827,5.86,9.59,0.0,0.0
27,East New York,0.015881,0.016815,16.00,11.31,0.0,0.0
28,East New York/Pennsylvania Avenue,0.016094,0.010602,7.44,5.00,0.0,0.0
25,East Flatbush/Farragut,0.024266,0.066698,12.75,12.75,0.0,0.0


In [106]:
b_tip_zones[['PU_Zone', 'avg_tip_PU', 'avg_tip_DO', 'max_tip_PU', 'max_tip_DO', 'min_tip_PU', 'min_tip_DO']].tail()

,PU_Zone,avg_tip_PU,avg_tip_DO,max_tip_PU,max_tip_DO,min_tip_PU,min_tip_DO
23,Downtown Brooklyn/MetroTech,0.858856,0.599005,44.20,44.20,0.0,0.0
16,Cobble Hill,0.933518,0.904904,24.20,24.20,0.0,0.0
17,Columbia Street,1.004309,1.132857,22.42,22.42,0.0,0.0
22,DUMBO/Vinegar Hill,1.034065,0.742216,20.20,17.74,0.0,0.0
51,Red Hook,1.122146,0.899356,32.20,40.20,0.0,0.0


In [107]:
# Bronx
bronx_tips.groupby(['PU_Zone', 'DO_Zone'])[['fare_amount','tip_amount']].max().sort_values(by='tip_amount', ascending=False).head()  

,,fare_amount,tip_amount
PU_Zone,DO_Zone,,
Van Nest/Morris Park,Van Nest/Morris Park,450.0,45.00
Co-Op City,Co-Op City,100.0,30.45
Riverdale/North Riverdale/Fieldston,Riverdale/North Riverdale/Fieldston,125.0,24.76
Mount Hope,Morrisania/Melrose,74.4,23.05
West Farms/Bronx River,West Farms/Bronx River,89.8,22.78


In [108]:
br_tip_pu = bronx_tips.groupby('PU_Zone')['tip_amount'].agg(avg_tip_PU='mean', max_tip_PU='max', min_tip_PU='min').reset_index() # avg, min, max tip by pickup zone
br_tip_do = bronx_tips.groupby('DO_Zone')['tip_amount'].agg(avg_tip_DO='mean', max_tip_DO='max', min_tip_DO='min').reset_index() # avg, min, max tip by dropoff zone
br_tip_zones = pd.merge(br_tip_pu, br_tip_do, left_on='PU_Zone', right_on='DO_Zone', how='inner').sort_values(by=['avg_tip_PU','avg_tip_DO'], ascending=True) # merge on zones
br_tip_zones[['PU_Zone', 'avg_tip_PU', 'avg_tip_DO', 'max_tip_PU', 'max_tip_DO', 'min_tip_PU', 'min_tip_DO']].head()

,PU_Zone,avg_tip_PU,avg_tip_DO,max_tip_PU,max_tip_DO,min_tip_PU,min_tip_DO
9,Crotona Park,0.000000,0.000000,0.0,0.0,0.0,0.0
34,Van Cortlandt Park,0.000000,0.000000,0.0,0.0,0.0,0.0
29,Schuylerville/Edgewater Park,0.000000,0.012136,0.0,5.0,0.0,0.0
4,Bronxdale,0.000000,0.023973,0.0,5.0,0.0,0.0
23,Norwood,0.007623,0.005780,3.4,4.0,0.0,0.0


In [109]:
br_tip_zones[['PU_Zone', 'avg_tip_PU', 'avg_tip_DO', 'max_tip_PU', 'max_tip_DO', 'min_tip_PU', 'min_tip_DO']].tail()

,PU_Zone,avg_tip_PU,avg_tip_DO,max_tip_PU,max_tip_DO,min_tip_PU,min_tip_DO
20,Morrisania/Melrose,0.174186,0.225752,16.19,23.05,0.0,0.0
37,West Concourse,0.200502,0.138882,21.30,21.30,0.0,0.0
5,City Island,0.484524,0.482923,20.35,20.35,0.0,0.0
28,Riverdale/North Riverdale/Fieldston,0.529225,0.414719,24.76,24.76,0.0,0.0
8,Country Club,0.685312,0.786944,10.00,10.00,0.0,0.0


In [110]:
# Staten Island
staten_island_tips.groupby(['PU_Zone', 'DO_Zone'])[['fare_amount','tip_amount']].max().sort_values(by='tip_amount', ascending=False).head()  

,,fare_amount,tip_amount
PU_Zone,DO_Zone,,
South Beach/Dongan Hills,New Dorp/Midland Beach,145.89,29.38
Oakwood,Oakwood,140.00,28.20
Charleston/Tottenville,Charleston/Tottenville,130.00,20.00
Grymes Hill/Clifton,Grymes Hill/Clifton,40.00,12.30
Saint George/New Brighton,Heartland Village/Todt Hill,47.10,10.00


In [111]:
si_tip_pu = staten_island_tips.groupby('PU_Zone')['tip_amount'].agg(avg_tip_PU='mean', max_tip_PU='max', min_tip_PU='min').reset_index() # avg, min, max tip by pickup zone
si_tip_do = staten_island_tips.groupby('DO_Zone')['tip_amount'].agg(avg_tip_DO='mean', max_tip_DO='max', min_tip_DO='min').reset_index() # avg, min, max tip by dropoff zone
si_tip_zones = pd.merge(si_tip_pu, si_tip_do, left_on='PU_Zone', right_on='DO_Zone', how='inner').sort_values(by=['avg_tip_PU','avg_tip_DO'], ascending=True) # merge on zones
si_tip_zones[['PU_Zone', 'avg_tip_PU', 'avg_tip_DO', 'max_tip_PU', 'max_tip_DO', 'min_tip_PU', 'min_tip_DO']].head()

,PU_Zone,avg_tip_PU,avg_tip_DO,max_tip_PU,max_tip_DO,min_tip_PU,min_tip_DO
0,Arden Heights,0.0,0.0,0.0,0.0,0.0,0.0
1,Arrochar/Fort Wadsworth,0.0,0.0,0.0,0.0,0.0,0.0
9,Port Richmond,0.0,0.0,0.0,0.0,0.0,0.0
10,Rossville/Woodrow,0.0,0.0,0.0,0.0,0.0,0.0
13,Stapleton,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
si_tip_zones[['PU_Zone', 'avg_tip_PU', 'avg_tip_DO', 'max_tip_PU', 'max_tip_DO', 'min_tip_PU', 'min_tip_DO']].tail()

,PU_Zone,avg_tip_PU,avg_tip_DO,max_tip_PU,max_tip_DO,min_tip_PU,min_tip_DO
5,Heartland Village/Todt Hill,2.000,5.000000,10.00,10.00,0.0,0.0
4,Grymes Hill/Clifton,2.050,6.150000,12.30,12.30,0.0,0.0
12,South Beach/Dongan Hills,2.092,0.226061,29.38,5.46,0.0,0.0
8,Oakwood,5.640,28.200000,28.20,28.20,0.0,28.2
3,Charleston/Tottenville,20.000,20.000000,20.00,20.00,20.0,20.0


> Whether tipping behaviors seem constant within their respective categories:
* Do expensive neighbourhoods have higher tips compared to the rest?
* Which areas have the most consistent tipping behaviors

*Note: Using a % calculation saved into a function in the data_utils pyscript*
* **'constant_tips'**
  * Calculates the percentage of rides in their respective categories are above the general average tip amount 
    * **borough_tips**:(e.g., `manhattan_tips`, `queens_tips`)
    * The 3 different neighborhood 'tips' df (e.g., `m_exp_tips`, `m_avg_tips`, `m_cheap_tips`)

In [113]:
# Manhattan
print(constant_tips(manhattan_tips, m_exp_tips, m_avg_tips, m_cheap_tips))

Expensive Neighborhoods: 30.41 %
Average Neighborhoods: 4.97 %
Cheap Neighborhoods: 48.37 %


In [114]:
# Queens
print(constant_tips(queens_tips, q_exp_tips, q_avg_tips, q_cheap_tips))

Expensive Neighborhoods: 44.8 %
Average Neighborhoods: 5.2 %
Cheap Neighborhoods: 82.64 %


In [115]:
# Brooklyn
print(constant_tips(brooklyn_tips, b_exp_tips, b_avg_tips, b_cheap_tips))

Expensive Neighborhoods: 4.64 %
Average Neighborhoods: 0.0 %
Cheap Neighborhoods: 94.35 %


In [116]:
# Bronx
print(constant_tips(bronx_tips, br_exp_tips, br_avg_tips, br_cheap_tips))

Expensive Neighborhoods: 1.23 %
Average Neighborhoods: 0.01 %
Cheap Neighborhoods: 99.49 %


In [117]:
# Staten Island
print(constant_tips(staten_island_tips, si_exp_tips, si_avg_tips, si_cheap_tips))

Expensive Neighborhoods: 28.57 %
Average Neighborhoods: 95.19 %
Cheap Neighborhoods: 100.0 %


> It seems that amount being tipped isn't relative/constant to its categorized zone
* In fact, it had the opposite effect
  * Instead of more expensive neighborhoods having tipped more, the cheaper and average neighborhoods tipped more 
  * The standard deviation shows that the cheaper and average neighborhoods' tipping was most consistent 
  * Cheaper fares --> more tip, Pricier fares --> less tip
  

> Consider looking at airports? Do they get tipped more on average?

In [118]:
# checking the number of airport-related rides in each borough
# using the merged_cleaned df to get the overall rides involving airports
merged_tips = borough_tip_cleaned(merged_cleaned)
airport_rides = merged_tips[(merged_tips['PU_service_zone'] == 'Airports') | (merged_tips['DO_service_zone'] == 'Airports') | (merged_tips['Airport_fee'] > 0)]  

In [119]:
airport_rides.shape[0]/merged_tips.shape[0] * 100  # percentage of airport-related rides in the overall dataset

9.196033749545098

In [120]:
airport_rides['tip_amount'].agg(['mean', 'median', 'max', 'min'])  # average, median, max, min tip amounts for airport-related rides

mean        9.77066
median     10.65000
max       157.80000
min         0.00000
Name: tip_amount, dtype: float64

In [121]:
merged_tips['tip_amount'].agg(['mean', 'median', 'max', 'min'])  # average, median, max, min tip amounts for all rides

mean        3.032312
median      2.350000
max       295.800000
min         0.000000
Name: tip_amount, dtype: float64

> On average, and judging by the median, airport rides do get more tips
* Most likely due to services such as:
  * Time sensitivity
  * Taking care of luggage

> Looking at the percentage of rides with no tips

In [122]:
dfs = [merged_tips,  manhattan_tips, queens_tips, brooklyn_tips, bronx_tips, staten_island_tips]
names = ['Overall', 'Manhattan', 'Queens', 'Brooklyn', 'Bronx', 'Staten Island']

# loop to calculate the percentage of rides with zero tip amount for each borough, then print the results
for df, name in zip(dfs, names):
    perc = df[df['tip_amount'] == 0].shape[0]/df.shape[0] * 100
    print(f'{name} : {round(perc, 2)} %')

Overall : 35.75 %
Manhattan : 32.27 %
Queens : 68.7 %
Brooklyn : 92.51 %
Bronx : 98.79 %
Staten Island : 89.62 %


> Seems like Bronx has the biggest %, while Manhattan has the smallest % out of the boroughs

#### **Key Findings Pt. 2:**

**Tip Amounts:**
* Manhattan & Queens have the highest average tips out of all the boroughs
* Average tip amounts:
  * Manhattan: $2.41
  * Queens: $2.83
  * Brooklyn: $0.31
  * Bronx: $0.07
  * Staten Island: 1.20
*generally seems like tipping is on the rather low side, with people in NYC either tipping the extremes or none at all*

**Neighborhood Segmentation:**
* It seems that amount being tipped isn't relative/constant to its categorized zone
* Instead of more expensive neighborhoods having tipped more, the cheaper and average neighborhoods tipped more 
* Cheaper fares --> more tip, Pricier fares --> less tip
  * Could be that customers tend to tip more when the fare is already cheaper (they feel more “giving”), which could explain why for all the boroughs, cheap neighborhoods are always the higher percentage
* Manhattan: 
  * Expensive neighborhoods consistently tip 30.41% in the range of their respective category
  * Whereas cheap neighborhoods have the higher percentage 48.37%
* Queens:
  * Expensive neighborhoods consistently tip 44.8% in the range of their respective category
  * Whereas cheap neighborhoods have the higher percentage 82.64%
* Brooklyn: 
  * Its cheap neighborhoods are consistent with 94.35%, the rest of the neighborhood categories are extremely low
* Bronx:
  * Has one of the highest percentage of the cheap neighborhoods at 99.49%
    * Is supportive of the pt.1 findings that Bronx tends to have a cheaper range
* Staten Island:
  * Its expensive neighborhoods consistently tip 28.57% of the time
  * Whereas the average neighborhoods 95.19%
  * Has a full 100% for cheap neighborhoods
  * But keep in mind Staten Island has the lowest amount of records, the percentages may be high due to the smaller sample size
* Overall, it seems Brooklyn and Bronx have the most consistent tipping behavior in their cheapest neighborhoods, whereas for expensive and average neighborhoods it seems to vary a lot

**Airport-related Rides:**
* 9.20% of the dataset are airport related
* Tip amounts:
  * Mean: $9.77, Median: $10.65, Max: $157.80, Min: $0
* Compared to the overall rides:
  * Mean: $3.03, Median: $2.35, Max: $295.8, Min: $0
* On average, and judging by the median, airport rides do get more tips
  * Most likely due to services such as:
    * Time sensitivity
    * Taking care of luggage
    * Supports that more service-related rides do get tipped more on average

**No Tip Rides:**
* Overall : 35.75 %
* Manhattan : 32.27 %, Queens : 68.7 %, Brooklyn : 92.51 %, Bronx : 98.79 %, Staten Island : 89.62 %
* Bronx has the biggest %, while Manhattan has the smallest % out of the boroughs

---
#### **2. Time Segmentation & Tipping Behavior**
* Let's take a closer look at whether there are datetime trends:
  * Weekends (people could be going out more)
  * Maybe there are activities after working hours (events, personal outings)
  * Time of day: morning, afternoon, night
* Will mostly be conducting aggregations

In [155]:
# lets create bin to categorize the datetime data
# basic morning, afternoon, evening, night bins
def time_bins(col):
    hour = col.hour
    if 5 <= hour < 12: # 5 AM to 11:59 AM
        return 'Morning'
    elif 12 <= hour < 17: # 12 PM to 4:59 PM
        return 'Afternoon'
    elif  17 <= hour < 21: # 5 PM to 8:59 PM
        return 'Evening'
    else:
        return 'Night'

# creating columns for time bins in the merged_tips dataframe
merged_tips['PU_time_bins'] = merged_tips['PU_datetime'].apply(time_bins)
merged_tips['DO_time_bins'] = merged_tips['DO_datetime'].apply(time_bins)

# rearranging the columns to have the datetime columns at the beginning for better readability
merged_tips_datetime = merged_tips[['PU_datetime', 'DO_datetime','PU_time_bins', 'DO_time_bins','tip_amount', 'fare_amount']].copy()


In [156]:
# sorting by the largest tip amounts to see what types of rides are most likely to have the largest tips
merged_tips_datetime.sort_values(by=['tip_amount'], ascending=False).head()

,PU_datetime,DO_datetime,PU_time_bins,DO_time_bins,tip_amount,fare_amount
2638916,2025-05-26 00:22:11,2025-05-26 00:22:15,Night,Night,295.80,985.0
2589997,2025-05-25 12:59:31,2025-05-25 12:59:35,Afternoon,Afternoon,187.75,750.0
282861,2025-05-03 23:19:43,2025-05-03 23:19:47,Night,Night,169.80,565.0
1931328,2025-05-18 23:32:51,2025-05-18 23:32:57,Night,Night,166.00,435.0
2090416,2025-05-20 16:22:06,2025-05-20 16:26:20,Afternoon,Afternoon,157.80,525.0


In [157]:
# lets check how many rides have the same PU and DO time bins
(merged_tips_datetime[merged_tips_datetime['PU_time_bins'] == merged_tips_datetime['DO_time_bins']].shape[0] / merged_tips_datetime.shape[0]) * 100  # percentage of rides with the same PU and DO time bins

94.76988085079637

> around 95% of the rides have the same time bins, the other 5% most likely took time place in between the bin switch *ex: 4:50-5 pm would be afternoon - evening*
* with that assumption, I will be dismissing the DO column

In [158]:
# will be choosing to group by the pickup time bins, since it is more likely that the pickup time will influence the tip amount
merged_tips_datetime.groupby('PU_time_bins')['tip_amount'].agg(['mean', 'median', 'max', 'min']).reset_index().sort_values(by='mean', ascending=False)

,PU_time_bins,mean,median,max,min
0,Afternoon,3.322883,2.52,187.75,0.0
1,Evening,3.150349,2.61,100.00,0.0
2,Morning,2.823755,2.00,132.50,0.0
3,Night,2.761223,2.00,295.80,0.0


> The average values show that most tips are given around the afternoon and evening, wheread morning and night tend to have slightly lower tips
* Afternoon & Night have higher max tips

In [159]:
# checking how many of each bin
merged_tips_datetime['PU_time_bins'].value_counts()


PU_time_bins
Afternoon    1152011
Evening      1096771
Night        1020859
Morning       898830
Name: count, dtype: int64

> Rides during the afternoon and evening seem to have the most amount 

In [160]:
# lets check the percentiles & distribution
merged_tips_datetime.groupby('PU_time_bins')['tip_amount'].describe().reset_index().sort_values(by=['mean', 'std'])

,PU_time_bins,count,mean,std,min,25%,50%,75%,max
3,Night,1020859.0,2.761223,3.772699,0.0,0.0,2.00,3.99,295.80
2,Morning,898830.0,2.823755,3.823748,0.0,0.0,2.00,3.79,132.50
1,Evening,1096771.0,3.150349,3.814848,0.0,0.0,2.61,4.30,100.00
0,Afternoon,1152011.0,3.322883,4.210878,0.0,0.0,2.52,4.35,187.75


> Night seems to have the lowest amount of variability with its 3.77 standard deviation, but it also has the lowest average tips
* its max amount is the highest, and its 75th percentile is slightly lower in comparison to the rest


In [ ]:
# further segment into weekdays and weekends for any insights
merged_tips_datetime['PU_is_weekend'] = merged_tips_datetime['PU_datetime'].dt.dayofweek >= 5  # 5 and 6 are saturday and sunday

# if True, then it is a weekend ride, otherwise it is a weekday ride
(merged_tips_datetime[merged_tips_datetime['PU_is_weekend'] == True].shape[0] / merged_tips_datetime.shape[0]) * 100  # percentage of weekend rides

29.145626777780148

> only 29% of rides are weekends

In [166]:
# for weekends
merged_tips_datetime[merged_tips_datetime['PU_is_weekend'] == True].groupby('PU_time_bins')['tip_amount'].agg(['count', 'mean', 'median', 'max', 'min']).reset_index().sort_values(by='mean', ascending=False)

,PU_time_bins,count,mean,median,max,min
0,Afternoon,335935,3.003728,2.15,187.75,0.0
1,Evening,284136,2.810557,2.08,98.23,0.0
2,Morning,204300,2.582040,1.82,132.50,0.0
3,Night,390556,2.366814,1.12,169.80,0.0


In [165]:
# for weekdays
merged_tips_datetime[merged_tips_datetime['PU_is_weekend'] == False].groupby('PU_time_bins')['tip_amount'].agg(['count','mean', 'median', 'max', 'min']).reset_index().sort_values(by='mean', ascending=False)

,PU_time_bins,count,mean,median,max,min
0,Afternoon,816076,3.454261,2.66,157.8,0.0
1,Evening,812635,3.269157,2.85,100.0,0.0
3,Night,630303,3.005612,2.31,295.8,0.0
2,Morning,694530,2.894857,2.11,112.0,0.0


> As a whole, trips on the weekdays tip more on average with more variability, why could that be?

#### **Key Findings**
---